<div style="font-variant: small-caps; 
      font-weight: normal; 
      font-size: 40px; 
      text-align: center; 
      padding: 15px; 
      margin: 10px;">
  Chatbot - Génératif - Agnostique
  </div> 
  
  
<div style="font-variant: small-caps; 
      font-weight: normal; 
      font-size: 30px; 
      text-align: center; 
      padding: 15px; 
      margin: 10px;">
  M2DS 2019
  </div> 
  

  <div style="font-variant: small-caps; 
      font-weight: normal; 
      font-size: 20px; 
      text-align: center; 
      padding: 15px;">
  Jean-baptiste Aujogue
  </div>

In [1]:
from __future__ import unicode_literals, print_function, division
import os
from io import open
import unicodedata
import string
import re
import random
import pickle
import copy
# package a installer d'abord avec anaconda
#import spacy

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
from torch.utils.data import DataLoader

# package a installer d'abord avec anaconda
import gensim
from gensim.models import KeyedVectors

#import nltk
#nltk.download()
#from nltk.tokenize import sent_tokenize, word_tokenize

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np

import unidecode
from unidecode import unidecode

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
import time
import math
import re
import json

In [3]:
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

USE_CUDA = torch.cuda.is_available()
print(USE_CUDA)
device = torch.device("cuda" if USE_CUDA else "cpu")

True


In [4]:
import sys
sys.path.append('C:\\Users\\Jb\\Desktop\\NLP\\chatNLP')

# 1 Training data preparation

### 1.1 Simple formatting of training dialogues

In [5]:
# -------------------- enlèvement des stopwords ---------------------------------------
def TrimWordsSentence(sentence, stopwords):
    '''Remove stopwords from a sentence'''
    resultwords = [word for word in sentence if word.lower() not in stopwords]
    return resultwords

def TrimWordsDialogue(dialogue, stopwords):
    '''Remove stopwords from user utterances in a dialogue'''
    for pair in dialogue: 
        pair[0] = TrimWordsSentence(pair[0], stopwords)
        #pair[1] = pair[1].strip()
    return dialogue

def TrimWords(dialogues, stopwords):
    '''Remove stopwords from user utterances in a list of dialogues'''
    return [TrimWordsDialogue(dialogue, stopwords) for dialogue in dialogues ]

    


# --------------------------- Normalisation -------------------------------
def normalizeString(s):
    '''Remove rare symbols from a string'''
    def unicodeToAscii(s):
        """Turn a Unicode string to plain ASCII, thanks to http://stackoverflow.com/a/518232/2809427"""
        return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')
 
    s = unicodeToAscii(s.lower().strip())
    #s = re.sub(r"[^a-zA-Z0-9?&\%\-\_]+", r" ", s) 
    s = re.sub("\(", r" ( ", s)
    s = re.sub("\)", r" ) ", s)
    s = re.sub(r"\.", r" . ", s)
    s = re.sub(r",", r" , ", s)
    s = re.sub(r"!", r" ! ", s)
    s = re.sub(r":", r" : ", s)
    s = re.sub(r"-", r" - ", s)
    s = re.sub(r"'", r" ' ", s)
    s = re.sub(r";", r" ; ", s)
    s = re.sub(r' +', r' ', s).strip()
    return s 



#--------------------- import des dialogues --------------------
def importDialogues(fichier, limite = None):
    '''Import a textfile containing dialogues and returns a list, each element 
       corresponding to a dialogue and also being under the form of a list, with 
       each element being a list of two elements : an element giving a user 
       utterance and another element giving the bot response. Both elements are 
       normalized strings.
       Ex. The dialogue :
       
               hi    hello what can i help you with today
               can you book a table    i m on it
               
       now becomes :
       
              [['hi', 'hello what can i help you with today'], 
               ['can you book a table', 'i m on it']]
               
       Lines corresponding to user utterance with no bot response are discarted.
    '''
    def cleanS(s):
        cleans = normalizeString(s).split(' ')
        cleans = [word for word in cleans if word.strip() != '']
        return cleans
    dialogues_import = open(fichier, encoding='utf-8').read().strip().split('\n\n')
    dialogues = []
    for i, d in enumerate(dialogues_import):
        dialogue = []
        lines = d.split('\n')
        for l in lines:
            if len(l.split('\t')) == 2 :
                pair = [cleanS(s) for s in l.split('\t')]
                dialogue.append(pair)
            elif len(l.split('\t')) == 3 :
                pair = [cleanS(s) for s in l.split('\t')[:2]]
                dialogue.append(pair)
        dialogues.append(dialogue)
        if limite is not None and i == limite -1 :
            break

    return dialogues






#------------------ Dictionnaire des mots variables -----------------------------
def motVar(file):
    '''Applies to the Master's program dataset.
       Import the collection of pairs token-content for a set of variable words.
    '''
    lines = open(file, encoding='utf-8').read().strip().split('\n')
    motsVar = {}
    for l in lines :
        cle, valeur = l.split('\t')
        motsVar[cle.lower()] = valeur
    return motsVar




# ----------------------- Création de la liste des dialogues--------------------
def prepareData(opt):
    '''Import dialogue from text file and apply some formatting operations,
       as described in the functions 
               - importDialogues
               - modify
               - TrimWords
               - filterDialogues
    '''
    dialogues = importDialogues(fichier = opt['fichier'], 
                                limite = opt['limite'])
    dialogues = modify(dialogues) if opt['modify'] else dialogues
    dialogues = TrimWords(dialogues, opt['stopwords']) # on enlève les stopwords de chaque question
    print(" %s dialogues ..." % len(dialogues))
    print(dialogues[0])
    if opt['filtre'] :
        #for pair in [pair for pair in pairs if not filterPair(pair)]:
        #    print('%s (%d) -> %s (%d)' % (pair[0],len(pair[0].split()),pair[1],len(pair[1].split())))  
        dialogues = filterDialogues(dialogues, opt['max_length'])
        print('')
        print("... reduced to %s dialogues" % len(dialogues))

    return dialogues

In [6]:
motsVar = motVar('C:\\Users\\Jb\\Desktop\\NLP\\data\\Conversations_M2DS\\chatbot-M2-DS-Variables.txt')

Turn each dialogue of the corpus into torch variables

In [7]:
def variableFromSentence(lang, sentence, rand = 0): # sentence = [str]
    indexes=[]
    unknowns = 0
    for word in sentence:
        p = random.random()
        if word not in lang.word2index.keys() and 'UNK' in lang.word2index.keys() :
            #pass
            indexes.append(lang.word2index['UNK'])
        elif p >= rand :
            indexes.append(lang.word2index[word])
        elif p < rand :
            e = random.choice([1, 2])
            if e == 1 :  # doesn't put any word
                pass
            elif e == 2 and 'UNK' in lang.word2index.keys() :# hide word with UNK_Token
                indexes.append(lang.word2index['UNK'])
    indexes.append(lang.word2index['EOS'])                                
    result = Variable(torch.LongTensor([[i] for i in indexes]))
    return result

def variableFromDialogue(lang, dialogue, rand = 0): # sentence = [str]
    result = []
    for paire in dialogue :
        el1 = variableFromSentence(lang, paire[0], rand = rand)
        el2 = variableFromSentence(lang, paire[1], rand = 0)
        result.append([el1, el2])
    return result

def variableFromAllDialogues(lang, dialogues, rand = 0): # sentence = [str]
    result = []
    for dialogue in dialogues :
        result.append(variableFromDialogue(lang, dialogue, rand))
    return result

### 1.2 Language class

In [8]:
class Lang:
    def __init__(self, name, init = 3):
        self.name = name
        if init == 0 :
            self.word2index = {}
            self.word2count = {}
            self.index2word = {}
            self.n_words = 0
        elif init == 3 :
            self.word2index = {"SOS": 0, "EOS": 1, "UNK": 2}
            self.word2count = {"SOS": 0, "EOS": 0, "UNK": 0}
            self.index2word = {0: "SOS", 1: "EOS", 2: "UNK"}
            self.n_words = 3

        
    def addWord(self, word):
        '''Add a word to the language'''
        if word not in self.word2index:
            if word.strip() != '' :
                self.word2index[word] = self.n_words
                self.word2count[word] = 1
                self.index2word[self.n_words] = word
                self.n_words += 1
        else:
            self.word2count[word] += 1
            
            
    def addSentence(self, sentence):
        '''Add to the language all words of a sentence'''
        if type(sentence) == str :
            for word in nltk.word_tokenize(sentence): # sentence.split()
                self.addWord(word)
        elif type(sentence) == list :
            for word in sentence:
                self.addWord(word)            
            
            
    def addDescriptions(self, descriptions, lvl = 1):
        '''Add to the language all words contained into : either all user utterances 
          (if i = 0) or all bot utterances (if i = 1), of a list of dialogues'''
        for description in descriptions :
            
            # si la description est une seule ligne
            if type(description) == str :
                    try :
                        if lvl == 2 :
                            for sentence in nltk.sent_tokenize(description) :
                                self.addSentence(sentence)
                        else :
                            self.addSentence(description)
                    except IndexError:
                        print("Problem with {}".format(description))
                        
            # si la description est une liste de lignes
            elif type(description) == list :
                for line in description:
                    try :
                        self.addSentence(line)
                    except IndexError:
                        print("Problem with {}".format(line))
                              
            # sinon
            else :
                print("Problem with {}".format(description))
            
            
    def addDialogues(self, dialogues, i):
        '''Add to the language all words contained into : either all user utterances 
          (if i = 0) or all bot utterances (if i = 1), of a list of dialogues'''
        for dialogue in dialogues :
            for pair in dialogue:
                try :
                    self.addSentence(pair[i])
                except IndexError:
                    print("Problem with {}".format(pair))

In [9]:
def generateLanguages(dialogues):
    '''Generate three languages classes out of a list of dialogues :
            - input_lang containing the user's vocabulary
            - output lang containing the bot vocabulary
            - output_sentence_lang containing the bot answers as words of a vocabulary
    '''
    lang = Lang('M2DS')
    
    lang.addDialogues(dialogues, 0)
    lang.addDialogues(dialogues, 1)
    print("Mots comptés :")
    print(lang.name, lang.n_words)
    
    return lang


def ajout(dialogues, lang, i= 1):
    '''addDialogues method of the Lang class with prints.'''
    lang.addDialogues(dialogues, i)
    print(lang.name, lang.n_words)
    return lang 


def ajoutSentences(dialogues, sentences_lang, i = 1) :
    '''Add sentences as words to a given language'''
    for dialogue in dialogues :
        for pair in dialogue :
            try :
                sentences_lang.addWord(pair[i])
            except IndexError:
                print("Problem with {}".format(pair))
                
    return sentences_lang

## 1.3 Master's program dataset

In [10]:
MAX_LENGTH = 100
max_length = MAX_LENGTH
stopwords = []

Master_train =   {'fichier': 'C:\\Users\\Jb\\Desktop\\NLP\\data\\Conversations_M2DS\\Conversations_M2DS_2019_final\\ChatbotDS_Train.tsv', #dialog-babi-task1-API-calls-trn.txt
                   'modify' : False,
                   'stopwords' : stopwords,
                   'max_length' : MAX_LENGTH ,
                   'limite' : None,
                   'filtre' : False}

dialogues_Master = prepareData(Master_train)

Master_test  =   {'lang1': 'lang_client',
                   'lang2' : 'lang_agent',
                   'fichier': 'C:\\Users\\Jb\\Desktop\\NLP\\data\\Conversations_M2DS\\Conversations_M2DS_2019_final\\ChatbotDS_Test.tsv', #dialog-babi-task1-API-calls-trn.txt
                   'modify' : False,
                   'stopwords' : stopwords,
                   'max_length' : MAX_LENGTH ,
                   'limite' : None,
                   'reverse' : False,
                   'filtre' : False}

dialogues_Master_test = prepareData(Master_test)

 100000 dialogues ...
[[['quel', 'est', 'l', "'", 'enseignant', 'de', 'l', "'", 'ue', 'de', 'pgm', '?'], ['l', "'", 'enseignant', 'de', 'modeles', 'graphiques', 'probabilistes', 'est', 'pr', '.', 'profpgm']], [['ca', 'va', 'merci', 'j', "'", 'ai', 'besoin', 'de', 'rien'], ['d', "'", 'accord', ',', 'j', "'", 'espere', 'avoir', 'correctement', 'repondu', 'a', 'vos', 'questions', '.']], [['que', 'faut', 'il', 'lire', 'en', 'statistique', 'inferentielle', '?'], ['vous', 'devriez', 'prendre', 'contact', 'avec', 'l', "'", 'enseignant', 'de', 'l', "'", 'ue', 'et', 'lui', 'demander']], [['je', 'viens', 'de', 'l', "'", 'etranger'], ['d', "'", 'accord', ',', 'comment', 'puis', '-', 'je', 'vous', 'aider', '?']], [['quel', 'est', 'le', 'cout', 'du', 'master', '?'], ['vous', 'devriez', 'poser', 'la', 'question', 'au', 'service', 'de', 'scolarite', '.']], [['c', "'", 'est', 'quand', 'la', 'fin', 'des', 'cours', '?'], ['aux', 'alentours', 'du', 'mois', 'de', 'mars', '.']], [['ca', 'va', 'merci', 'j',

In [11]:
clean_sentences = [sentence for sentence in list(set([' '.join(pair[0]) for diag in dialogues_Master for pair in diag]))]
print(len(clean_sentences))

1364


In [12]:
lang_M2DS = generateLanguages(dialogues_Master)

Mots comptés :
M2DS 1291


In [13]:
# compute ONCE
#with open(r'C:\\Users\\Jb\\Desktop\\NLP\\saves\\lang_M2DS_2019_final.file', 'wb') as fileObject :
#    pickle.dump(lang_M2DS, fileObject)

# 1062 / 1272 / 1291 mots comptés

In [14]:
def importLang(name):
    with open('C:\\Users\\Jb\\Desktop\\NLP\\saves\\lang\\'+name+'.file', 'rb') as fil :
        lang = pickle.load(fil)
    return lang

In [15]:
#lang_M2DS = importLang('lang_M2DS_2019_final') # 1062 / 1272 / 1291

# 2 Word Embedding Pretraining

## 2.1 Models

In [16]:
from chatNLP.models.Word_Embedding import CBOW, SkipGram, WordEmbeddingTrainer

### 2.1.1 Cbow

In [16]:
cbow = CBOW(device,
            lang = lang_M2DS,
            context_size = 5,
            embedding_dim = 100,
            hidden_dim = 100)
cbow = cbow.to(device)

In [ ]:
cbow.load_state_dict(torch.load('C:\\Users\\Jb\\Desktop\\NLP\\saves\\models\\M2DS_cbow.pth'))

### 2.1.2 SkipGram

**Data**

In [18]:
def flatten(description) :
    '''Baisse le nombre de niveaux de 1 dans la description'''
    flatten = []
    for line in description :
        flatten += line
    return flatten


def generateNgrams(descriptions, context_size = 5, word_first = True) :
    '''descriptions = [[str]]'''
    data = []
    for description in descriptions :
        lines = flatten(description)
        line =  flatten(lines)
        line = ['SOS' for i in range(context_size)] + line + ['EOS' for i in range(context_size)] 
        for i in range(context_size, len(line) - context_size):
            context = line[i-context_size : i] + line[i+1 : i+context_size+1]
            target = [line[i]]
            if word_first :
                data.append((target, context))
            else :
                data.append((context, target))
    return data

**Model**

In [17]:
skipgram = SkipGram(device, 
                    lang = lang_M2DS, 
                    context_size = 5, 
                    embedding_dim = 100, 
                    hidden_dim = 150)
skipgram = skipgram.to(device)

In [18]:
skipgram.load_state_dict(torch.load('C:\\Users\\Jb\\Desktop\\NLP\\saves\\models\\M2DS_2019_final_skipgram_100.pth'))

**Training**

In [19]:
word_embedding_trainer = WordEmbeddingTrainer(device = device)

In [22]:
for alpha in [0.01, 0.005, 0.0025, 0.001] :
    for i in range(20) :
        Ngrams = generateNgrams(dialogues_Master[5000*i:5000*(i+1)], context_size = 5, word_first = True)
        word_embedding_trainer.train(skipgram, Ngrams, n_iters = 5000, learning_rate=alpha)

0m 0s (- 0m 21s) (100 2%) 7.1594 9.98
0m 0s (- 0m 22s) (200 4%) 7.0452 9.82
0m 1s (- 0m 21s) (300 6%) 6.9597 9.68
0m 1s (- 0m 21s) (400 8%) 6.8505 9.45
0m 2s (- 0m 21s) (500 10%) 6.7443 9.51
0m 2s (- 0m 20s) (600 12%) 6.6424 9.61
0m 3s (- 0m 20s) (700 14%) 6.5610 9.62
0m 3s (- 0m 19s) (800 16%) 6.4343 9.56
0m 4s (- 0m 19s) (900 18%) 6.4177 9.61
0m 4s (- 0m 18s) (1000 20%) 6.3094 9.58
0m 5s (- 0m 18s) (1100 22%) 6.2583 9.54
0m 5s (- 0m 17s) (1200 24%) 6.1976 9.43
0m 6s (- 0m 17s) (1300 26%) 6.1901 9.66
0m 6s (- 0m 16s) (1400 28%) 6.1105 9.57
0m 7s (- 0m 16s) (1500 30%) 6.0057 9.48
0m 7s (- 0m 16s) (1600 32%) 5.9809 9.46
0m 8s (- 0m 15s) (1700 34%) 5.8727 9.50
0m 8s (- 0m 15s) (1800 36%) 5.9425 9.57
0m 8s (- 0m 14s) (1900 38%) 5.8525 9.39
0m 9s (- 0m 14s) (2000 40%) 5.8764 9.68
0m 9s (- 0m 13s) (2100 42%) 5.8636 9.53
0m 10s (- 0m 13s) (2200 44%) 5.7840 9.41
0m 10s (- 0m 12s) (2300 46%) 5.7784 9.44
0m 11s (- 0m 12s) (2400 48%) 5.7474 9.43
0m 11s (- 0m 11s) (2500 50%) 5.7906 9.48
0m 12s (-

0m 3s (- 0m 20s) (700 14%) 5.0561 8.75
0m 3s (- 0m 19s) (800 16%) 4.8894 8.61
0m 4s (- 0m 19s) (900 18%) 4.9069 8.50
0m 4s (- 0m 19s) (1000 20%) 4.6914 8.27
0m 5s (- 0m 18s) (1100 22%) 4.9471 8.71
0m 5s (- 0m 18s) (1200 24%) 5.0160 8.70
0m 6s (- 0m 17s) (1300 26%) 4.8829 8.63
0m 6s (- 0m 17s) (1400 28%) 5.0645 8.79
0m 7s (- 0m 16s) (1500 30%) 4.8150 8.40
0m 7s (- 0m 16s) (1600 32%) 4.7054 8.37
0m 8s (- 0m 15s) (1700 34%) 4.7635 8.39
0m 8s (- 0m 15s) (1800 36%) 4.7259 8.62
0m 9s (- 0m 14s) (1900 38%) 4.7044 8.28
0m 9s (- 0m 14s) (2000 40%) 4.7505 8.41
0m 9s (- 0m 13s) (2100 42%) 4.8755 8.72
0m 10s (- 0m 13s) (2200 44%) 5.0200 8.81
0m 10s (- 0m 12s) (2300 46%) 4.7184 8.38
0m 11s (- 0m 12s) (2400 48%) 4.9043 8.70
0m 11s (- 0m 11s) (2500 50%) 4.8362 8.48
0m 12s (- 0m 11s) (2600 52%) 4.8654 8.78
0m 12s (- 0m 10s) (2700 54%) 4.9942 8.97
0m 13s (- 0m 10s) (2800 56%) 4.7940 8.25
0m 13s (- 0m 9s) (2900 57%) 4.7298 8.39
0m 14s (- 0m 9s) (3000 60%) 4.9214 8.71
0m 14s (- 0m 8s) (3100 62%) 4.7805 8

0m 6s (- 0m 17s) (1300 26%) 4.5776 8.19
0m 6s (- 0m 16s) (1400 28%) 4.4845 8.03
0m 7s (- 0m 16s) (1500 30%) 4.3223 7.92
0m 7s (- 0m 16s) (1600 32%) 4.2603 7.81
0m 8s (- 0m 15s) (1700 34%) 4.6256 8.33
0m 8s (- 0m 15s) (1800 36%) 4.6110 8.27
0m 8s (- 0m 14s) (1900 38%) 4.5013 8.02
0m 9s (- 0m 14s) (2000 40%) 4.3780 8.07
0m 9s (- 0m 13s) (2100 42%) 4.4514 7.95
0m 10s (- 0m 13s) (2200 44%) 4.4938 8.07
0m 10s (- 0m 12s) (2300 46%) 4.5416 8.38
0m 11s (- 0m 12s) (2400 48%) 4.5946 8.24
0m 11s (- 0m 11s) (2500 50%) 4.3505 7.75
0m 12s (- 0m 11s) (2600 52%) 4.3954 8.04
0m 12s (- 0m 10s) (2700 54%) 4.5276 8.12
0m 13s (- 0m 10s) (2800 56%) 4.7808 8.69
0m 13s (- 0m 9s) (2900 57%) 4.4101 7.97
0m 14s (- 0m 9s) (3000 60%) 4.3978 7.97
0m 14s (- 0m 8s) (3100 62%) 4.4204 8.21
0m 15s (- 0m 8s) (3200 64%) 4.5013 8.28
0m 15s (- 0m 8s) (3300 66%) 4.5139 8.17
0m 16s (- 0m 7s) (3400 68%) 4.5528 8.21
0m 16s (- 0m 7s) (3500 70%) 4.5080 8.11
0m 16s (- 0m 6s) (3600 72%) 4.5392 8.38
0m 17s (- 0m 6s) (3700 74%) 4.540

0m 8s (- 0m 14s) (1900 38%) 4.3775 8.08
0m 9s (- 0m 13s) (2000 40%) 4.4017 7.92
0m 9s (- 0m 13s) (2100 42%) 4.4008 7.99
0m 10s (- 0m 13s) (2200 44%) 4.3468 8.16
0m 10s (- 0m 12s) (2300 46%) 4.1667 7.78
0m 11s (- 0m 12s) (2400 48%) 4.5333 8.18
0m 11s (- 0m 11s) (2500 50%) 3.9663 7.35
0m 12s (- 0m 11s) (2600 52%) 4.2298 7.93
0m 12s (- 0m 10s) (2700 54%) 4.2475 7.94
0m 13s (- 0m 10s) (2800 56%) 4.2008 7.84
0m 13s (- 0m 9s) (2900 57%) 4.2457 8.04
0m 14s (- 0m 9s) (3000 60%) 4.3257 7.97
0m 14s (- 0m 9s) (3100 62%) 4.2536 7.84
0m 15s (- 0m 8s) (3200 64%) 4.1494 7.45
0m 15s (- 0m 8s) (3300 66%) 4.4451 8.25
0m 16s (- 0m 7s) (3400 68%) 4.4235 8.21
0m 16s (- 0m 7s) (3500 70%) 4.3650 8.07
0m 16s (- 0m 6s) (3600 72%) 4.0337 7.51
0m 17s (- 0m 6s) (3700 74%) 4.2600 7.71
0m 17s (- 0m 5s) (3800 76%) 4.2932 7.87
0m 18s (- 0m 5s) (3900 78%) 4.4843 8.33
0m 19s (- 0m 4s) (4000 80%) 4.3821 8.19
0m 19s (- 0m 4s) (4100 82%) 4.1174 7.67
0m 20s (- 0m 3s) (4200 84%) 4.2081 7.79
0m 20s (- 0m 3s) (4300 86%) 4.380

0m 11s (- 0m 12s) (2400 48%) 3.9475 7.82
0m 11s (- 0m 11s) (2500 50%) 4.1049 7.66
0m 12s (- 0m 11s) (2600 52%) 4.3373 8.14
0m 12s (- 0m 10s) (2700 54%) 3.9633 7.46
0m 13s (- 0m 10s) (2800 56%) 4.0356 7.18
0m 13s (- 0m 9s) (2900 57%) 4.0953 7.79
0m 13s (- 0m 9s) (3000 60%) 4.2054 7.85
0m 14s (- 0m 8s) (3100 62%) 4.1204 7.80
0m 14s (- 0m 8s) (3200 64%) 4.2569 8.13
0m 15s (- 0m 7s) (3300 66%) 4.1097 7.85
0m 15s (- 0m 7s) (3400 68%) 4.1384 7.37
0m 16s (- 0m 6s) (3500 70%) 4.1052 7.78
0m 16s (- 0m 6s) (3600 72%) 4.1633 7.84
0m 17s (- 0m 6s) (3700 74%) 4.1667 7.97
0m 17s (- 0m 5s) (3800 76%) 4.1449 7.90
0m 18s (- 0m 5s) (3900 78%) 3.8902 7.56
0m 18s (- 0m 4s) (4000 80%) 4.0855 7.89
0m 19s (- 0m 4s) (4100 82%) 4.0471 7.82
0m 19s (- 0m 3s) (4200 84%) 4.1001 7.86
0m 20s (- 0m 3s) (4300 86%) 3.9562 7.64
0m 20s (- 0m 2s) (4400 88%) 4.2155 7.92
0m 20s (- 0m 2s) (4500 90%) 4.4333 8.34
0m 21s (- 0m 1s) (4600 92%) 4.5058 8.28
0m 21s (- 0m 1s) (4700 94%) 4.2111 7.77
0m 22s (- 0m 0s) (4800 96%) 4.1564 

...

0m 14s (- 0m 9s) (3000 60%) 3.7596 7.39
0m 14s (- 0m 9s) (3100 62%) 3.6081 7.28
0m 15s (- 0m 8s) (3200 64%) 3.7527 7.60
0m 15s (- 0m 8s) (3300 66%) 3.5942 7.32
0m 16s (- 0m 7s) (3400 68%) 3.8776 7.86
0m 16s (- 0m 7s) (3500 70%) 3.6543 7.57
0m 17s (- 0m 6s) (3600 72%) 3.4553 6.98
0m 17s (- 0m 6s) (3700 74%) 3.9067 7.86
0m 17s (- 0m 5s) (3800 76%) 3.7532 7.54
0m 18s (- 0m 5s) (3900 78%) 3.6957 7.51
0m 18s (- 0m 4s) (4000 80%) 3.6645 7.39
0m 19s (- 0m 4s) (4100 82%) 3.5418 7.36
0m 19s (- 0m 3s) (4200 84%) 3.5642 7.30
0m 20s (- 0m 3s) (4300 86%) 3.5884 7.06
0m 20s (- 0m 2s) (4400 88%) 3.8091 7.42
0m 21s (- 0m 2s) (4500 90%) 3.8404 7.57
0m 21s (- 0m 1s) (4600 92%) 3.6742 7.26
0m 22s (- 0m 1s) (4700 94%) 3.9474 7.81
0m 22s (- 0m 0s) (4800 96%) 3.7020 7.69
0m 23s (- 0m 0s) (4900 98%) 3.4887 7.14
0m 23s (- 0m 0s) (5000 100%) 3.5245 6.90
0m 0s (- 0m 23s) (100 2%) 3.6487 7.16
0m 0s (- 0m 23s) (200 4%) 3.5515 7.33
0m 1s (- 0m 22s) (300 6%) 3.7324 7.52
0m 1s (- 0m 21s) (400 8%) 3.8403 7.78
0m 2s (

0m 18s (- 0m 7s) (3500 70%) 3.6110 7.21
0m 19s (- 0m 7s) (3600 72%) 3.9640 7.81
0m 19s (- 0m 6s) (3700 74%) 3.9986 7.96
0m 20s (- 0m 6s) (3800 76%) 3.5769 7.10
0m 21s (- 0m 5s) (3900 78%) 3.6039 7.28
0m 21s (- 0m 5s) (4000 80%) 3.4820 7.22
0m 22s (- 0m 4s) (4100 82%) 3.7157 7.56
0m 22s (- 0m 4s) (4200 84%) 3.7624 7.52
0m 23s (- 0m 3s) (4300 86%) 3.8353 7.45
0m 23s (- 0m 3s) (4400 88%) 3.6339 7.40
0m 24s (- 0m 2s) (4500 90%) 3.7642 7.59
0m 24s (- 0m 2s) (4600 92%) 3.8708 7.67
0m 25s (- 0m 1s) (4700 94%) 3.5469 7.25
0m 25s (- 0m 1s) (4800 96%) 3.6382 7.10
0m 26s (- 0m 0s) (4900 98%) 3.5187 7.08
0m 26s (- 0m 0s) (5000 100%) 3.6497 7.11
0m 0s (- 0m 29s) (100 2%) 3.6026 7.32
0m 1s (- 0m 29s) (200 4%) 3.5099 7.10
0m 1s (- 0m 28s) (300 6%) 3.5628 7.09
0m 2s (- 0m 27s) (400 8%) 3.5369 7.42
0m 2s (- 0m 26s) (500 10%) 3.8354 7.59
0m 3s (- 0m 27s) (600 12%) 3.8025 7.41
0m 4s (- 0m 27s) (700 14%) 3.4754 7.12
0m 5s (- 0m 27s) (800 16%) 3.6907 7.69
0m 5s (- 0m 26s) (900 18%) 3.8151 7.86
0m 6s (- 0m 

0m 20s (- 0m 5s) (4000 80%) 3.6025 7.28
0m 21s (- 0m 4s) (4100 82%) 3.5368 7.30
0m 21s (- 0m 4s) (4200 84%) 3.7034 7.37
0m 22s (- 0m 3s) (4300 86%) 3.6323 7.34
0m 23s (- 0m 3s) (4400 88%) 3.8263 7.64
0m 23s (- 0m 2s) (4500 90%) 3.5854 7.44
0m 24s (- 0m 2s) (4600 92%) 3.6472 7.30
0m 24s (- 0m 1s) (4700 94%) 3.4531 6.62
0m 25s (- 0m 1s) (4800 96%) 3.6653 7.36
0m 25s (- 0m 0s) (4900 98%) 3.6198 7.28
0m 26s (- 0m 0s) (5000 100%) 3.8120 7.42
0m 0s (- 0m 31s) (100 2%) 3.7274 8.03
0m 1s (- 0m 30s) (200 4%) 3.5048 7.06
0m 1s (- 0m 29s) (300 6%) 3.5894 7.30
0m 2s (- 0m 27s) (400 8%) 3.3555 6.86
0m 3s (- 0m 27s) (500 10%) 3.6877 7.49
0m 3s (- 0m 27s) (600 12%) 3.7644 7.48
0m 4s (- 0m 26s) (700 14%) 3.5439 7.06
0m 4s (- 0m 26s) (800 16%) 4.0254 7.88
0m 5s (- 0m 24s) (900 18%) 3.5609 7.37
0m 6s (- 0m 24s) (1000 20%) 3.7567 7.49
0m 6s (- 0m 23s) (1100 22%) 3.6788 7.58
0m 7s (- 0m 22s) (1200 24%) 3.5866 7.30
0m 7s (- 0m 22s) (1300 26%) 3.7863 7.75
0m 8s (- 0m 21s) (1400 28%) 3.7586 7.41
0m 8s (- 0m 

0m 21s (- 0m 2s) (4500 90%) 3.4318 7.11
0m 22s (- 0m 1s) (4600 92%) 3.8793 7.87
0m 22s (- 0m 1s) (4700 94%) 3.9009 8.01
0m 23s (- 0m 0s) (4800 96%) 3.7366 7.66
0m 23s (- 0m 0s) (4900 98%) 3.6211 7.42
0m 24s (- 0m 0s) (5000 100%) 3.7660 7.55
0m 0s (- 0m 23s) (100 2%) 3.3664 7.06
0m 0s (- 0m 23s) (200 4%) 3.9198 7.57
0m 1s (- 0m 22s) (300 6%) 3.5387 6.93
0m 1s (- 0m 21s) (400 8%) 3.6943 7.55
0m 2s (- 0m 21s) (500 10%) 3.5564 7.47
0m 2s (- 0m 20s) (600 12%) 3.8122 7.80
0m 3s (- 0m 20s) (700 14%) 3.5455 7.40
0m 3s (- 0m 19s) (800 16%) 3.4197 6.94
0m 4s (- 0m 19s) (900 18%) 3.6127 7.31
0m 4s (- 0m 18s) (1000 20%) 3.7678 7.65
0m 5s (- 0m 18s) (1100 22%) 3.7676 7.64
0m 5s (- 0m 18s) (1200 24%) 3.5832 7.27
0m 6s (- 0m 17s) (1300 26%) 3.8684 8.06
0m 6s (- 0m 17s) (1400 28%) 3.6287 7.55
0m 7s (- 0m 16s) (1500 30%) 3.6009 7.18
0m 7s (- 0m 16s) (1600 32%) 3.5517 7.05
0m 8s (- 0m 15s) (1700 34%) 3.3168 6.96
0m 8s (- 0m 15s) (1800 36%) 4.1436 7.89
0m 9s (- 0m 14s) (1900 38%) 3.6856 7.26
0m 9s (- 0m 

0m 26s (- 0m 0s) (5000 100%) 3.5623 7.49
0m 0s (- 0m 23s) (100 2%) 3.6847 7.50
0m 0s (- 0m 23s) (200 4%) 3.7571 7.57
0m 1s (- 0m 23s) (300 6%) 3.6961 7.63
0m 1s (- 0m 22s) (400 8%) 3.6184 7.30
0m 2s (- 0m 23s) (500 10%) 3.8745 7.70
0m 3s (- 0m 22s) (600 12%) 3.5855 7.09
0m 3s (- 0m 22s) (700 14%) 3.6325 7.52
0m 4s (- 0m 21s) (800 16%) 3.5979 7.23
0m 4s (- 0m 20s) (900 18%) 3.7526 7.28
0m 5s (- 0m 20s) (1000 20%) 3.5998 7.10
0m 5s (- 0m 19s) (1100 22%) 3.9905 8.02
0m 6s (- 0m 19s) (1200 24%) 3.5715 7.34
0m 6s (- 0m 18s) (1300 26%) 3.7272 7.64
0m 7s (- 0m 18s) (1400 28%) 3.6764 7.40
0m 7s (- 0m 18s) (1500 30%) 3.9377 7.63
0m 8s (- 0m 17s) (1600 32%) 3.5365 7.22
0m 8s (- 0m 17s) (1700 34%) 3.6799 7.38
0m 9s (- 0m 17s) (1800 36%) 3.3782 7.17
0m 10s (- 0m 16s) (1900 38%) 3.4938 7.13
0m 10s (- 0m 16s) (2000 40%) 3.6105 7.11
0m 11s (- 0m 15s) (2100 42%) 3.3589 7.25
0m 11s (- 0m 15s) (2200 44%) 3.5112 7.00
0m 12s (- 0m 14s) (2300 46%) 3.4606 7.03
0m 12s (- 0m 13s) (2400 48%) 3.4769 7.18
0m 13s

In [ ]:
#word_embedding_trainer.train(skipgram, Ngrams, n_iters = 100000, learning_rate=0.01)
#word_embedding_trainer.train(skipgram, Ngrams, n_iters = 100000, learning_rate=0.005)
#word_embedding_trainer.train(skipgram, Ngrams, n_iters = 100000, learning_rate=0.0025)
#word_embedding_trainer.train(skipgram, Ngrams, n_iters = 100000, learning_rate=0.001)

In [23]:
torch.save(skipgram.state_dict(), 'C:\\Users\\Jb\\Desktop\\NLP\\saves\\models\\M2DS_2019_final_skipgram_100.pth')

# 3  Sentence Embedding Pretraining

In [ ]:
from chatNLP.modules.Encoder_Words_Recurrent import RecurrentWordsEncoder
from chatNLP.modules.Decoder_Words           import WordsDecoder
from chatNLP.modules.Decoder_Words_Attn      import AttnWordsDecoder

## 3.1 Data

In [ ]:
clean_sentences = [sentence for sentence in list(set([' '.join(s) for diag in dialogues_Master for pair in diag for s in pair]))]
clean_sentences = [s.split() for s in clean_sentences]
print(len(clean_sentences))
print(max([len(s) for s in clean_sentences]))

In [ ]:
clean_sentences_var = [variableFromSentence(lang_M2DS, s, rand = 0) for s in clean_sentences]

In [ ]:
clean_sentences_var_025 = [variableFromSentence(lang_M2DS, s, rand = 0.25) for s in clean_sentences]

## 3.2 Model

In [ ]:
class AutoEncoderTrainer(object):
    def __init__(self, 
                 device,
                 criterion = nn.NLLLoss(), 
                 optimizer = optim.SGD, 
                 clipping = 10, 
                 print_every=100):
        
        # relevant quantities
        self.device = device
        self.criterion = criterion.to(device)
        self.optimizer = optimizer
        self.clip = clipping
        self.print_every = print_every# timer
        
        
    def asMinutes(self, s):
        m = math.floor(s / 60)
        s -= m * 60
        return '%dm %ds' % (m, s)


    def timeSince(self, since, percent):
        now = time.time()
        s = now - since
        es = s / (percent)
        rs = es - s
        return '%s (- %s)' % (self.asMinutes(s), self.asMinutes(rs))
        
        
    def distance(self, agent_outputs, target_answer) :
        """ Compute cumulated error between predicted output and ground answer."""
        loss = 0
        loss_diff_mots = 0
        agent_outputs_length = len(agent_outputs)
        target_length = len(target_answer)
        Max = max(agent_outputs_length, target_length)
        Min = min(agent_outputs_length, target_length)   
        for i in range(Min):
            agent_output = agent_outputs[i]
            target_word = target_answer[i]
            #print(agent_output.size(), target_answer.size())
            factor = (1 + Max - Min) if i == Min -1 else 1
            loss += factor * self.criterion(agent_output, target_word)
            topv, topi = agent_output.data.topk(1)
            ni = topi[0][0]
            if ni != target_word.data[0]:
                loss_diff_mots += 1
        loss_diff_mots += Max - Min
        return loss, loss_diff_mots
        
        
    def trainLoop(self, encoder, decoder, sentence, optimizer):
        """Performs a training loop, with forward pass and backward pass for gradient optimisation."""
        optimizer.zero_grad()
        sentence_length = len(sentence)
        sentence = sentence.to(self.device)
        word_hiddens, last_hidden = encoder(sentence)
        output_sentence, log_probas = decoder(query_words     = word_hiddens, 
                                              query_vector    = None, 
                                              decision_vector = last_hidden.view(1, 1, -1), 
                                              target_answer   = sentence)

        loss, loss_diff_mots = self.distance(log_probas, sentence)        
        loss.backward()
        _ = torch.nn.utils.clip_grad_norm_(encoder.parameters(), self.clip)
        _ = torch.nn.utils.clip_grad_norm_(decoder.parameters(), self.clip)
        optimizer.step()
        return loss.data[0] / sentence_length , loss_diff_mots
        
        
    def train(self, encoder, decoder, sentences, n_iters = 10000, learning_rate = 0.01, dic = None, return_errors = False):
        """Performs training over a given dataset and along a specified amount of loops."""
        start = time.time()
        optimizer = self.optimizer([param for param in encoder.parameters() if param.requires_grad == True] + 
                                   [param for param in decoder.parameters() if param.requires_grad == True], 
                                   lr=learning_rate)
        print_loss_total = 0  
        print_loss_diff_mots_total = 0
        if return_errors : errors = []
        for iter in range(1, n_iters + 1):
            j = int(random.choice(dic)) if dic is not None else int(random.choice(range(len(sentences))))
            sentence = sentences[j]
            loss, loss_diff_mots = self.trainLoop(encoder, decoder, sentence, optimizer)
            if return_errors and loss_diff_mots > 0 and j not in errors : errors.append(j)
            # quantité d'erreurs sur la réponse i
            print_loss_total += loss
            print_loss_diff_mots_total += loss_diff_mots       
            if iter % self.print_every == 0:
                print_loss_avg = print_loss_total / self.print_every
                print_loss_diff_mots_avg = print_loss_diff_mots_total / self.print_every
                print_loss_total = 0
                print_loss_diff_mots_total = 0
                print('%s (%d %d%%) %.4f %.2f' % (self.timeSince(start, iter / n_iters),
                                             iter, iter / n_iters * 100, print_loss_avg, print_loss_diff_mots_avg))
        if return_errors : return errors

In [ ]:
embedding = skipgram.embedding
for param in embedding.parameters() : param.requires_grad = False


encoder = RecurrentWordsEncoder(device, 
                                embedding = embedding, 
                                hidden_dim = 100, 
                                n_layers = 1, 
                                dropout = 0.15)


decoder = AttnWordsDecoder(device,
                           embedding = embedding,                                   
                           hidden_dim = encoder.output_dim,
                           n_layers = 1,
                           tracking_dim = 0,
                           dropout = 0.15,
                           tf_ratio = 1.0,
                           bound = 75) 


encoder = encoder.to(device)
decoder = decoder.to(device)

In [ ]:
autoencoder_trainer = AutoEncoderTrainer(device = device,
                                         criterion = nn.NLLLoss(), 
                                         optimizer = optim.SGD, 
                                         clipping = 10,
                                         print_every = 100)

## 3.3 Training

In [ ]:
encoder.train()
decoder.train()
autoencoder_trainer.train(encoder, decoder, sentences = clean_sentences_var, n_iters = 10000, learning_rate = 0.05)

In [ ]:
encoder.train()
decoder.train()
autoencoder_trainer.train(encoder, decoder, sentences = clean_sentences_var, n_iters = 10000, learning_rate = 0.02)

In [ ]:
encoder.train()
decoder.train()
autoencoder_trainer.train(encoder, decoder, sentences = clean_sentences_var, n_iters = 10000, learning_rate = 0.005)

In [ ]:
encoder.train()
decoder.train()
autoencoder_trainer.train(encoder, decoder, sentences = clean_sentences_var, n_iters = 10000, learning_rate = 0.005)

In [ ]:
torch.save(encoder.state_dict(), 'C:\\Users\\Jb\\Desktop\\NLP\\saves\\models\\M2DS_2019_final_encoder_150.pth')
torch.save(decoder.state_dict(), 'C:\\Users\\Jb\\Desktop\\NLP\\saves\\models\\M2DS_2019_final_decoder_150.pth')

***
# 4 + 5 + 6 Chatbot implementation

In [19]:
from chatNLP.models.Chatbot import Chatbot, CreateBot, BotTrainer

## 6.1 Data

In [67]:
dialogues_Master_var = variableFromAllDialogues(lang_M2DS, dialogues_Master, rand = 0)

In [20]:
dialogues_Master_test_var = variableFromAllDialogues(lang_M2DS, dialogues_Master_test, rand = 0)

In [ ]:
bound = max([max([len(el[1]) for el in dial]) for dial in dialogues_Master_var])
bound

## 6.2 Model

In [21]:
# entrainement avec balayage de chaque dialogue en entier 

class BotTrainer(object):
    def __init__(self, 
                 device,
                 criterion = nn.NLLLoss(), 
                 optimizer = optim.SGD, 
                 clipping = 10, 
                 print_every=100):
        
        # relevant quantities
        self.device = device
        self.criterion = criterion.to(device)
        self.optimizer = optimizer
        self.clip = clipping
        self.print_every = print_every# timer
        
        
    def asMinutes(self, s):
        m = math.floor(s / 60)
        s -= m * 60
        return '%dm %ds' % (m, s)


    def timeSince(self, since, percent):
        now = time.time()
        s = now - since
        es = s / (percent)
        rs = es - s
        return '%s (- %s)' % (self.asMinutes(s), self.asMinutes(rs))
        
        
    def distance(self, agent_outputs, target_answer) :
        """ Compute cumulated error between predicted output and ground answer."""
        loss = 0
        loss_diff_mots = 0
        agent_outputs_length = len(agent_outputs)
        target_length = len(target_answer)
        Max = max(agent_outputs_length, target_length)
        Min = min(agent_outputs_length, target_length)   
        for i in range(Min):
            agent_output = agent_outputs[i]
            target_word = target_answer[i]
            #print(agent_output.size(), target_answer.size())
            factor = (1 + Max - Min) if i == Min -1 else 1
            loss += factor * self.criterion(agent_output, target_word)
            topv, topi = agent_output.data.topk(1)
            ni = topi[0][0]
            if ni != target_word.data[0]:
                loss_diff_mots += 1
        loss_diff_mots += Max - Min
        return loss, loss_diff_mots
        
        
    def trainLoop(self, agent, dialogue, target_answer, optimizer):
        """Performs a training loop, with forward pass and backward pass for gradient optimisation."""
        optimizer.zero_grad()
        query = dialogue[-1][0].to(self.device)
        target_answer = target_answer.to(self.device)
        answer, agent_outputs, attn1_w, attn2_w, _ = agent.answerTrain(dialogue, target_answer) 
        loss, loss_diff_mots = self.distance(agent_outputs, target_answer)
        loss.backward()
        _ = torch.nn.utils.clip_grad_norm_(agent.parameters(), self.clip)
        optimizer.step()
        return loss.data[0] / len(target_answer), loss_diff_mots
        
        
    def train(self, agent, dialogues, n_iters = 10000, learning_rate=0.01, dic = None, per_dialogue = False, return_errors = False):
        """Performs training over a given dataset and along a specified amount of loops."""
        if type(dialogues[0][0]) == list :
            debut = 0
            double = True
        else :
            debut = 1
            double = False
        start = time.time()
        optimizer = self.optimizer([param for param in agent.parameters() if param.requires_grad == True], lr=learning_rate)
        print_loss_total = 0  
        print_loss_diff_mots_total = 0
        if return_errors : errors = {}
        iter = 1
        while iter < n_iters :
            if dic is not None :
                j = int(random.choice(list(dic.keys())))
                training_dialogue = dialogues[j]
                i = random.choice(dic[j])
                partie_dialogue = training_dialogue[:i+1-debut]
                target_answer = training_dialogue[i][1] if double else training_dialogue[i]
                loss, loss_diff_mots = self.trainLoop(agent, partie_dialogue, target_answer, optimizer)
                if return_errors and loss_diff_mots > 0 :
                    if j not in list(errors.keys()) : errors[j] = []
                    if i not in errors[j] : errors[j].append(i)
                # quantité d'erreurs sur la réponse i
                print_loss_total += loss
                print_loss_diff_mots_total += loss_diff_mots 
                iter += 1
                if iter % self.print_every == 0:
                    print_loss_avg = print_loss_total / self.print_every
                    print_loss_diff_mots_avg = print_loss_diff_mots_total / self.print_every
                    print_loss_total = 0
                    print_loss_diff_mots_total = 0
                    print('%s (%d %d%%) %.4f %.2f' % (self.timeSince(start, iter / n_iters), iter, iter / n_iters * 100, 
                                                                  print_loss_avg, print_loss_diff_mots_avg))
            elif per_dialogue :
                j = int(random.choice(range(len(dialogues))))
                training_dialogue = dialogues[j]
                indices = list(range(debut, len(training_dialogue)))
                random.shuffle(indices)
                for i in indices :
                    partie_dialogue = training_dialogue[:i+1]
                    target_answer = training_dialogue[i][1] if double else training_dialogue[i]
                    loss, loss_diff_mots = self.trainLoop(agent, partie_dialogue, target_answer, optimizer)
                    if return_errors and loss_diff_mots > 0 :
                        if j not in list(errors.keys()) : errors[j] = []
                        if i not in errors[j] : errors[j].append(i)
                    # quantité d'erreurs sur la réponse i
                    print_loss_total += loss
                    print_loss_diff_mots_total += loss_diff_mots
                    iter += 1
                    if iter % self.print_every == 0:
                        print_loss_avg = print_loss_total / self.print_every
                        print_loss_diff_mots_avg = print_loss_diff_mots_total / self.print_every
                        print_loss_total = 0
                        print_loss_diff_mots_total = 0
                        print('%s (%d %d%%) %.4f %.2f' % (self.timeSince(start, iter / n_iters), iter, iter / n_iters * 100, 
                                                                      print_loss_avg, print_loss_diff_mots_avg))
            else :
                j = int(random.choice(range(len(dialogues))))
                training_dialogue = dialogues[j]
                i = random.choice(range(debut, len(training_dialogue)))
                partie_dialogue = training_dialogue[:i+1]
                target_answer = training_dialogue[i][1] if double else training_dialogue[i]
                loss, loss_diff_mots = self.trainLoop(agent, partie_dialogue, target_answer, optimizer)
                if return_errors and loss_diff_mots > 0 :
                    if j not in list(errors.keys()) : errors[j] = []
                    if i not in errors[j] : errors[j].append(i)
                # quantité d'erreurs sur la réponse i
                print_loss_total += loss
                print_loss_diff_mots_total += loss_diff_mots
                iter += 1
                if iter % self.print_every == 0:
                    print_loss_avg = print_loss_total / self.print_every
                    print_loss_diff_mots_avg = print_loss_diff_mots_total / self.print_every
                    print_loss_total = 0
                    print_loss_diff_mots_total = 0
                    print('%s (%d %d%%) %.4f %.2f' % (self.timeSince(start, iter / n_iters), iter, iter / n_iters * 100, 
                                                                  print_loss_avg, print_loss_diff_mots_avg))


        if return_errors : return errors
                
                
    def ErrorCount(self, agent, dialogues):
        bound = 10
        ERRORS = [0 for i in range(bound +1)]
        repartitionError = {}
        for i in range(bound +1) :
            repartitionError[i] = []
        liste = []
        for k, input_dialogue in enumerate(dialogues):
            for l in range(len(input_dialogue)):
                if len(input_dialogue[l][1])>0 :
                    dialogue = input_dialogue[:l+1]
                    #target_answer = variableFromSentence(agent.output_lang, input_dialogue[l][1])
                    target_answer = input_dialogue[l][1]
                    target_answer = target_answer.to(self.device)
                    answer, agent_outputs, attn1_w, attn2_w, _ = agent.answerTrain(dialogue)
                    loss, loss_diff_mots = self.distance(agent_outputs, target_answer)
                    if loss_diff_mots > bound :
                        ERRORS = ERRORS + [0 for i in range(loss_diff_mots - bound)]
                        for i in range(bound +1, loss_diff_mots +1) :
                            repartitionError[i] = []
                        bound  = loss_diff_mots
                    ERRORS[loss_diff_mots] += 1
                    if loss_diff_mots > 0 :
                        liste.append([k, l, loss_diff_mots])
        for triple in liste:
            repartitionError[triple[2]].append(triple[:2])
        print("The repartition of errors :", ERRORS)
        return repartitionError


    def DialoguesWithErrors(self, agent, dialogues) :
        '''Returns a dictionnary, with indices of dialogues and index of line in dialogue
           where a mistake was made.
        '''
        start = time.time()
        Sortie = {}
        L = len(dialogues)
        for i, dialogue in enumerate(dialogues) :
            errs = []
            for j in range(len(dialogue)) :
                target_answer = dialogue[j][1]
                target_answer = target_answer.to(self.device)
                answer, agent_outputs, attn1_w, attn2_w, _ = agent.answerTrain(dialogue[:j+1], target_answer)
                loss, loss_diff_mots = self.distance(agent_outputs, target_answer)
                if loss_diff_mots > 0 :
                    errs.append(j)
            if errs != []:
                Sortie[i] = errs
            if (i+1) % self.print_every == 0:
                print('%s (%d %d%%)' % (self.timeSince(start, (i+1) / L),
                                             (i+1), (i+1) / L * 100))
        return Sortie

In [28]:
# entrainement avec auteoencodage
class BotTrainer(object):
    def __init__(self, 
                 device,
                 criterion = nn.NLLLoss(), 
                 optimizer = optim.SGD,
                 autoencoding_ratio = 0.5,
                 clipping = 10, 
                 print_every=100):
        
        # relevant quantities
        self.device = device
        self.criterion = criterion.to(device)
        self.optimizer = optimizer
        self.aa = autoencoding_ratio
        self.clip = clipping
        self.print_every = print_every# timer
        
        
    def asMinutes(self, s):
        m = math.floor(s / 60)
        s -= m * 60
        return '%dm %ds' % (m, s)


    def timeSince(self, since, percent):
        now = time.time()
        s = now - since
        es = s / (percent)
        rs = es - s
        return '%s (- %s)' % (self.asMinutes(s), self.asMinutes(rs))
        
        
    def distance(self, agent_outputs, target_answer) :
        """ Compute cumulated error between predicted output and ground answer."""
        loss = 0
        loss_diff_mots = 0
        agent_outputs_length = len(agent_outputs)
        target_length = len(target_answer)
        Max = max(agent_outputs_length, target_length)
        Min = min(agent_outputs_length, target_length)   
        for i in range(Min):
            agent_output = agent_outputs[i]
            target_word = target_answer[i]
            #print(agent_output.size(), target_answer.size())
            factor = (1 + Max - Min) if i == Min -1 else 1
            loss += factor * self.criterion(agent_output, target_word)
            topv, topi = agent_output.data.topk(1)
            ni = topi[0][0]
            if ni != target_word.data[0]:
                loss_diff_mots += 1
        loss_diff_mots += Max - Min
        return loss, loss_diff_mots
        
        
    def trainLoop(self, agent, dialogue, target_answer, optimizer):
        """Performs a training loop, with forward pass and backward pass for gradient optimisation."""
        optimizer.zero_grad()
        query = dialogue[-1][0].to(self.device)
        target_answer = target_answer.to(self.device)
        answer, agent_outputs, attn1_w, attn2_w, autoencoder_outputs = agent.answerTrain(dialogue, target_answer) 
        loss1, loss_diff_mots1 = self.distance(agent_outputs, target_answer)       
        loss2, loss_diff_mots2 = self.distance(autoencoder_outputs, query) if autoencoder_outputs is not None else None, None
        loss = loss1 + self.aa * loss2 autoencoder_outputs is not None else loss1
        loss.backward()
        _ = torch.nn.utils.clip_grad_norm_(agent.parameters(), self.clip)
        optimizer.step()
        l1 = loss1.data[0] / len(target_answer)
        l2 = loss2.data[0] / len(query) if autoencoder_outputs is not None else None
        return l1, loss_diff_mots1, l2 , loss_diff_mots2
        
        
    def train(self, agent, dialogues, n_iters = 10000, learning_rate=0.01, dic = None, return_errors = False):
        """Performs training over a given dataset and along a specified amount of loops."""
        if type(dialogues[0][0]) == list :
            debut = 0
            double = True
        else :
            debut = 1
            double = False
        start = time.time()
        optimizer = self.optimizer([param for param in agent.parameters() if param.requires_grad == True], lr=learning_rate)
        print_loss1_total = 0
        print_loss2_total = 0
        print_loss_diff_mots1_total = 0
        print_loss_diff_mots2_total = 0
        if return_errors : errors = {}
        for iter in range(1, n_iters + 1):
            if dic is not None :
                j = int(random.choice(list(dic.keys())))
                training_dialogue = dialogues[j]
                i = random.choice(dic[j])
                partie_dialogue = training_dialogue[:i+1-debut]
                target_answer = training_dialogue[i][1] if double else training_dialogue[i]
            else :
                j = int(random.choice(range(len(dialogues))))
                training_dialogue = dialogues[j]
                i = random.choice(range(debut, len(training_dialogue)))
                partie_dialogue = training_dialogue[:i+1]
                target_answer = training_dialogue[i][1] if double else training_dialogue[i]

            loss1, loss_diff_mots1, loss2, loss_diff_mots2 = self.trainLoop(agent, partie_dialogue, target_answer, optimizer)
            if return_errors and loss_diff_mots1 > 0 :
                if j not in list(errors.keys()) : errors[j] = []
                if i not in errors[j] : errors[j].append(i)
            # quantité d'erreurs sur la réponse i
            print_loss1_total += loss1
            print_loss2_total += loss2
            print_loss_diff_mots1_total += loss_diff_mots1  
            print_loss_diff_mots2_total += loss_diff_mots2  
            if iter % self.print_every == 0:
                print_loss1_avg = print_loss1_total / self.print_every
                print_loss2_avg = print_loss2_total / self.print_every
                print_loss_diff_mots1_avg = print_loss_diff_mots1_total / self.print_every
                print_loss_diff_mots2_avg = print_loss_diff_mots2_total / self.print_every
                print_loss1_total = 0
                print_loss2_total = 0
                print_loss_diff_mots1_total = 0
                print_loss_diff_mots2_total = 0
                print('%s (%d %d%%) %.4f %.2f | %.4f %.2f' % (self.timeSince(start, iter / n_iters), iter, iter / n_iters * 100, 
                                                              print_loss1_avg, print_loss_diff_mots1_avg,
                                                              print_loss2_avg, print_loss_diff_mots2_avg))
        if return_errors : return errors
                
                
    def ErrorCount(self, agent, dialogues):
        bound = 10
        ERRORS = [0 for i in range(bound +1)]
        repartitionError = {}
        for i in range(bound +1) :
            repartitionError[i] = []
        liste = []
        for k, input_dialogue in enumerate(dialogues):
            for l in range(len(input_dialogue)):
                if len(input_dialogue[l][1])>0 :
                    dialogue = input_dialogue[:l+1]
                    #target_answer = variableFromSentence(agent.output_lang, input_dialogue[l][1])
                    target_answer = input_dialogue[l][1]
                    target_answer = target_answer.to(self.device)
                    answer, agent_outputs, attn1_w, attn2_w, _ = agent.answerTrain(dialogue)
                    loss, loss_diff_mots = self.distance(agent_outputs, target_answer)
                    if loss_diff_mots > bound :
                        ERRORS = ERRORS + [0 for i in range(loss_diff_mots - bound)]
                        for i in range(bound +1, loss_diff_mots +1) :
                            repartitionError[i] = []
                        bound  = loss_diff_mots
                    ERRORS[loss_diff_mots] += 1
                    if loss_diff_mots > 0 :
                        liste.append([k, l, loss_diff_mots])
        for triple in liste:
            repartitionError[triple[2]].append(triple[:2])
        print("The repartition of errors :", ERRORS)
        return repartitionError


    def DialoguesWithErrors(self, agent, dialogues) :
        '''Returns a dictionnary, with indices of dialogues and index of line in dialogue
           where a mistake was made.
        '''
        start = time.time()
        Sortie = {}
        L = len(dialogues)
        for i, dialogue in enumerate(dialogues) :
            errs = []
            for j in range(len(dialogue)) :
                target_answer = dialogue[j][1]
                target_answer = target_answer.to(self.device)
                answer, agent_outputs, attn1_w, attn2_w, _ = agent.answerTrain(dialogue[:j+1], target_answer)
                loss, loss_diff_mots = self.distance(agent_outputs, target_answer)
                if loss_diff_mots > 0 :
                    errs.append(j)
            if errs != []:
                Sortie[i] = errs
            if (i+1) % self.print_every == 0:
                print('%s (%d %d%%)' % (self.timeSince(start, (i+1) / L),
                                             (i+1), (i+1) / L * 100))
        return Sortie

In [22]:
trainer = BotTrainer(device = device,
                     criterion = nn.NLLLoss(), 
                     optimizer = optim.SGD,
                     #autoencoding_ratio = 0.5,
                     clipping = 10,
                     print_every = 100)

In [23]:
# query_vector <- decision_vector en WordDecoder

chatbot = CreateBot(  lang_M2DS,
                      embedding = skipgram.embedding,
                      hidden_dim = 75,
                      n_layers = 1, # doit être gardé à 1
                    
                      sentence_hidden_dim = 75,
                      hops = 3,
                      share = True, #False
                      transf = True,
                      dropout = 0.10,
                    
                      attn_decoder_n_layers = -1, # -1 : pas d'attention durant le décodage
                      language_model_n_layers = 0,
                      tf_ratio = 1.0,
                      bound = 75,
                      autoencoding = False,
                    
                      device = device
                     )

chatbot.nbParametres()

670741

In [35]:
chatbot.load_state_dict(torch.load('C:\\Users\\Jb\\Desktop\\NLP\\saves\\models\\M2DS_2019_final_chatbot_100_75_75_010_17.pth')) # chatbot_original / 26 / 39

In [43]:
# si on veut voir le pipeline du modèle
chatbot

Chatbot(
  (encoder): RecurrentWordsEncoder(
    (embedding): Embedding(1291, 100)
    (dropout): Dropout(p=0.1)
    (bigru): GRU(100, 75, bidirectional=True)
  )
  (attention): RecurrentHierarchicalAttention(
    (dropout): Dropout(p=0.1)
    (attn1): ModuleList(
      (0): AdditiveAttention(
        (attn_layer): Linear(in_features=300, out_features=150, bias=True)
        (attn_v): Linear(in_features=150, out_features=1, bias=False)
      )
      (1): AdditiveAttention(
        (attn_layer): Linear(in_features=300, out_features=150, bias=True)
        (attn_v): Linear(in_features=150, out_features=1, bias=False)
      )
      (2): AdditiveAttention(
        (attn_layer): Linear(in_features=300, out_features=150, bias=True)
        (attn_v): Linear(in_features=150, out_features=1, bias=False)
      )
    )
    (bigru): GRU(150, 75, bidirectional=True)
    (attn2): ModuleList(
      (0): AdditiveAttention(
        (attn_layer): Linear(in_features=300, out_features=150, bias=True)
    

## 6.3 Training

### 6.3.1 Standard training

In [44]:
chatbot.train()
trainer.train(chatbot, dialogues_Master_var, n_iters = 20000, per_dialogue = True, learning_rate=0.05)

0m 31s (- 104m 27s) (100 0%) 6.0390 14.74
1m 8s (- 112m 24s) (200 1%) 5.0853 13.40
1m 38s (- 107m 56s) (300 1%) 4.2667 12.55
2m 6s (- 103m 15s) (400 2%) 3.7172 10.59
2m 43s (- 106m 7s) (500 2%) 3.4454 11.68
3m 15s (- 105m 37s) (600 3%) 3.1664 9.87
3m 45s (- 103m 40s) (700 3%) 2.6337 7.52
4m 19s (- 103m 53s) (800 4%) 2.7174 8.36
4m 51s (- 103m 9s) (900 4%) 2.6302 8.58
5m 25s (- 103m 10s) (1000 5%) 2.5334 8.82
5m 55s (- 101m 56s) (1100 5%) 2.1286 7.61
6m 26s (- 100m 50s) (1200 6%) 2.4811 8.00
7m 3s (- 101m 26s) (1300 6%) 2.3022 7.66
7m 39s (- 101m 43s) (1400 7%) 1.9777 6.72
8m 13s (- 101m 27s) (1500 7%) 1.9716 6.64
8m 46s (- 101m 0s) (1600 8%) 1.9869 6.24
9m 21s (- 100m 42s) (1700 8%) 2.2244 8.05
9m 55s (- 100m 20s) (1800 9%) 1.8275 7.98
10m 29s (- 99m 56s) (1900 9%) 2.0028 6.19
11m 1s (- 99m 9s) (2000 10%) 1.6312 4.79
11m 31s (- 98m 13s) (2100 10%) 1.6957 6.20
12m 3s (- 97m 29s) (2200 11%) 1.6641 6.02
12m 33s (- 96m 37s) (2300 11%) 1.5998 4.87
13m 1s (- 95m 29s) (2400 12%) 1.6017 6.23
1

105m 26s (- 4m 23s) (19200 96%) 0.2410 1.50
106m 2s (- 3m 50s) (19300 96%) 0.1366 0.56
106m 38s (- 3m 17s) (19400 97%) 0.1230 0.48
107m 9s (- 2m 44s) (19500 97%) 0.2263 1.14
107m 37s (- 2m 11s) (19600 98%) 0.1084 0.32
108m 10s (- 1m 38s) (19700 98%) 0.1139 0.61
108m 44s (- 1m 5s) (19800 99%) 0.1162 0.37
109m 15s (- 0m 32s) (19900 99%) 0.1174 0.39
109m 49s (- 0m 0s) (20000 100%) 0.0996 0.42


In [45]:
torch.save(chatbot.state_dict(), 'C:\\Users\\Jb\\Desktop\\NLP\\saves\\models\\M2DS_2019_final_chatbot_100_75_75_010_1.pth')

In [46]:
chatbot.train()
errors = trainer.train(chatbot, dialogues_Master_var, n_iters = 20000, per_dialogue = True, learning_rate=0.05, return_errors = True)

0m 33s (- 112m 8s) (100 0%) 0.0737 0.28
1m 4s (- 106m 15s) (200 1%) 0.1292 0.50
1m 33s (- 102m 41s) (300 1%) 0.1013 0.40
2m 0s (- 98m 36s) (400 2%) 0.1037 0.44
2m 33s (- 99m 36s) (500 2%) 0.1398 0.56
3m 3s (- 98m 42s) (600 3%) 0.2207 1.28
3m 39s (- 100m 43s) (700 3%) 0.1001 0.31
4m 5s (- 98m 21s) (800 4%) 0.1121 0.36
4m 37s (- 98m 9s) (900 4%) 0.1536 0.48
5m 10s (- 98m 27s) (1000 5%) 0.1497 0.60
5m 38s (- 96m 52s) (1100 5%) 0.1355 0.39
6m 12s (- 97m 18s) (1200 6%) 0.1381 0.42
6m 44s (- 96m 52s) (1300 6%) 0.1836 0.78
7m 20s (- 97m 31s) (1400 7%) 0.1375 0.53
8m 1s (- 99m 0s) (1500 7%) 0.1348 0.63
8m 35s (- 98m 52s) (1600 8%) 0.0954 0.33
8m 59s (- 96m 52s) (1700 8%) 0.0756 0.33
9m 26s (- 95m 32s) (1800 9%) 0.1112 0.37
9m 57s (- 94m 56s) (1900 9%) 0.1105 0.60
10m 23s (- 93m 30s) (2000 10%) 0.0981 0.41
10m 48s (- 92m 11s) (2100 10%) 0.1158 0.33
11m 16s (- 91m 11s) (2200 11%) 0.1133 0.34
11m 53s (- 91m 28s) (2300 11%) 0.0869 0.52
12m 23s (- 90m 52s) (2400 12%) 0.0938 0.32
12m 51s (- 90m 0s) 

104m 50s (- 4m 22s) (19200 96%) 0.0335 0.10
105m 36s (- 3m 49s) (19300 96%) 0.0373 0.17
106m 7s (- 3m 16s) (19400 97%) 0.0529 0.21
106m 39s (- 2m 44s) (19500 97%) 0.0346 0.13
107m 22s (- 2m 11s) (19600 98%) 0.0729 0.60
107m 58s (- 1m 38s) (19700 98%) 0.0303 0.08
108m 47s (- 1m 5s) (19800 99%) 0.0545 0.20
109m 20s (- 0m 32s) (19900 99%) 0.0549 0.20
109m 55s (- 0m 0s) (20000 100%) 0.0364 0.13


In [47]:
# 9775
torch.save(chatbot.state_dict(), 'C:\\Users\\Jb\\Desktop\\NLP\\saves\\models\\M2DS_2019_final_chatbot_100_75_75_010_2.pth')

In [48]:
chatbot.eval()
z = trainer.ErrorCount(chatbot, dialogues_Master_test_var) # erreurs par réponse

The repartition of errors : [9775, 9, 219, 2, 28, 124, 40, 155, 97, 179, 772, 1085, 700, 662, 899, 692, 819, 1271, 553, 497, 168, 197, 3, 46, 79, 43, 550, 53, 27, 32, 0, 3, 1, 0, 1, 13, 0, 0, 0, 0, 2, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 15, 0, 0, 0, 0, 0, 6, 0, 0, 0, 35]


In [53]:
chatbot.train()
trainer.train(chatbot, dialogues_Master_var, n_iters = 5000, learning_rate=0.05, dic = errors)
trainer.train(chatbot, dialogues_Master_var, n_iters = 5000, per_dialogue = True, learning_rate=0.05)

0m 25s (- 21m 11s) (100 2%) 0.6235 0.62
1m 5s (- 26m 11s) (200 4%) 1.6466 1.84
1m 36s (- 25m 17s) (300 6%) 0.6267 0.70
2m 9s (- 24m 49s) (400 8%) 0.7218 1.02
2m 42s (- 24m 20s) (500 10%) 0.7678 0.93
3m 15s (- 23m 55s) (600 12%) 0.9338 1.16
3m 54s (- 23m 59s) (700 14%) 0.6453 0.75
4m 29s (- 23m 33s) (800 16%) 1.1264 1.62
4m 56s (- 22m 30s) (900 18%) 0.6044 0.79
5m 30s (- 22m 2s) (1000 20%) 0.9425 0.92
5m 59s (- 21m 15s) (1100 22%) 0.8160 1.35
6m 32s (- 20m 43s) (1200 24%) 0.6887 0.91
7m 11s (- 20m 26s) (1300 26%) 0.0182 0.05
7m 41s (- 19m 47s) (1400 28%) 0.2127 0.32
8m 15s (- 19m 16s) (1500 30%) 1.1444 1.35
8m 52s (- 18m 52s) (1600 32%) 0.4307 0.62
9m 22s (- 18m 12s) (1700 34%) 0.9643 1.17
9m 54s (- 17m 36s) (1800 36%) 0.4475 0.86
10m 25s (- 17m 0s) (1900 38%) 0.2840 0.44
10m 55s (- 16m 23s) (2000 40%) 0.2526 0.44
11m 29s (- 15m 52s) (2100 42%) 0.4322 0.81
12m 2s (- 15m 20s) (2200 44%) 0.0659 0.14
12m 34s (- 14m 45s) (2300 46%) 0.0183 0.06
13m 8s (- 14m 14s) (2400 48%) 0.4858 0.72
13m 4

### 6.3.2 Teacher Forcing Ratio = 0.5

In [25]:
chatbot.decoder.tf_ratio = 0.5

In [57]:
chatbot.train()
errors = trainer.train(chatbot, dialogues_Master_var, n_iters = 20000, per_dialogue = True, learning_rate=0.05, return_errors = True)

0m 24s (- 41m 4s) (100 1%) 0.3116 0.35
0m 58s (- 47m 53s) (200 2%) 0.3560 0.48
1m 35s (- 51m 40s) (300 3%) 0.4301 1.30
2m 13s (- 53m 17s) (400 4%) 0.2200 0.38
2m 38s (- 50m 6s) (500 5%) 0.5639 1.06
3m 15s (- 50m 58s) (600 6%) 0.5912 1.17
3m 49s (- 50m 54s) (700 7%) 0.5171 1.03
4m 19s (- 49m 46s) (800 8%) 0.0101 0.01
4m 58s (- 50m 21s) (900 9%) 0.4899 0.75
5m 29s (- 49m 27s) (1000 10%) 0.0573 0.08
6m 8s (- 49m 40s) (1100 11%) 0.6568 1.05
6m 40s (- 49m 0s) (1200 12%) 0.0118 0.02
7m 19s (- 49m 1s) (1300 13%) 0.1753 0.39
7m 53s (- 48m 27s) (1400 14%) 0.2588 0.26
8m 23s (- 47m 32s) (1500 15%) 0.5179 0.65
8m 58s (- 47m 7s) (1600 16%) 0.3030 0.30
9m 35s (- 46m 51s) (1700 17%) 0.1456 0.13
10m 13s (- 46m 35s) (1800 18%) 0.4059 0.99
10m 48s (- 46m 6s) (1900 19%) 0.1351 0.19
11m 15s (- 45m 3s) (2000 20%) 0.2348 0.55
11m 47s (- 44m 22s) (2100 21%) 0.7895 0.94
12m 18s (- 43m 39s) (2200 22%) 0.2440 0.72
12m 44s (- 42m 39s) (2300 23%) 0.1731 0.19
13m 15s (- 42m 0s) (2400 24%) 0.3756 0.20
13m 47s (- 4

In [58]:
# 10041
torch.save(chatbot.state_dict(), 'C:\\Users\\Jb\\Desktop\\NLP\\saves\\models\\M2DS_2019_final_chatbot_100_75_75_010_3.pth')

In [26]:
chatbot.eval()
z = trainer.ErrorCount(chatbot, dialogues_Master_test_var) # erreurs par réponse

The repartition of errors : [10041, 14, 42, 5, 0, 44, 110, 129, 101, 183, 1040, 487, 238, 546, 1630, 567, 1018, 1411, 245, 373, 433, 610, 15, 18, 106, 49, 263, 16, 0, 37, 1, 0, 0, 0, 6, 17, 0, 0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 35]


### 6.3.3 Random masking 25 % of corpus words

In [27]:
dialogues_Master_var_025 = variableFromAllDialogues(lang_M2DS, dialogues_Master, rand = 0.25)

In [28]:
chatbot.train()
errors = trainer.train(chatbot, dialogues_Master_var_025, n_iters = 20000, per_dialogue = True, learning_rate=0.05, return_errors = True)

0m 25s (- 85m 29s) (100 0%) 4.5761 3.88
0m 49s (- 82m 23s) (200 1%) 2.6355 2.98
1m 7s (- 73m 53s) (300 1%) 3.2997 4.88
1m 26s (- 70m 41s) (400 2%) 2.3500 3.44
1m 48s (- 70m 16s) (500 2%) 2.4425 3.80
2m 8s (- 68m 59s) (600 3%) 2.2887 3.38
2m 26s (- 67m 22s) (700 3%) 2.0975 2.96
2m 41s (- 64m 40s) (800 4%) 2.3636 3.10
3m 0s (- 63m 45s) (900 4%) 2.4988 3.09
3m 19s (- 63m 18s) (1000 5%) 1.7505 2.71
3m 38s (- 62m 32s) (1100 5%) 2.4163 3.06
3m 57s (- 62m 1s) (1200 6%) 3.0383 3.82
4m 20s (- 62m 28s) (1300 6%) 2.8229 4.53
4m 42s (- 62m 30s) (1400 7%) 2.4673 3.63
4m 59s (- 61m 37s) (1500 7%) 2.3202 2.96
5m 14s (- 60m 15s) (1600 8%) 2.2801 3.18
5m 31s (- 59m 25s) (1700 8%) 3.8288 5.98
5m 52s (- 59m 21s) (1800 9%) 3.1625 5.01
6m 10s (- 58m 46s) (1900 9%) 1.6757 2.46
6m 25s (- 57m 53s) (2000 10%) 2.3363 4.00
6m 41s (- 57m 3s) (2100 10%) 2.7109 4.06
7m 0s (- 56m 43s) (2200 11%) 2.0035 3.47
7m 20s (- 56m 27s) (2300 11%) 2.8869 3.73
7m 42s (- 56m 30s) (2400 12%) 2.1927 2.94
8m 1s (- 56m 8s) (2500 12%

64m 46s (- 2m 20s) (19300 96%) 1.9930 3.23
65m 7s (- 2m 0s) (19400 97%) 1.8859 3.17
65m 25s (- 1m 40s) (19500 97%) 2.4226 3.00
65m 44s (- 1m 20s) (19600 98%) 2.0259 2.83
66m 5s (- 1m 0s) (19700 98%) 1.7982 3.67
66m 25s (- 0m 40s) (19800 99%) 2.0387 3.73
66m 43s (- 0m 20s) (19900 99%) 2.1701 4.02
67m 4s (- 0m 0s) (20000 100%) 1.3994 2.55


In [29]:
torch.save(chatbot.state_dict(), 'C:\\Users\\Jb\\Desktop\\NLP\\saves\\models\\M2DS_2019_final_chatbot_100_75_75_010_4.pth')

In [30]:
chatbot.train()
trainer.train(chatbot, dialogues_Master_var_025, n_iters = 5000, learning_rate=0.05, dic = errors)
errors = trainer.train(chatbot, dialogues_Master_var_025, n_iters = 20000, per_dialogue = True, learning_rate=0.05, return_errors = True)

0m 17s (- 14m 41s) (100 2%) 2.7228 4.14
0m 34s (- 13m 58s) (200 4%) 3.6073 7.58
0m 53s (- 13m 56s) (300 6%) 3.3311 5.75
1m 9s (- 13m 24s) (400 8%) 3.0228 5.16
1m 28s (- 13m 13s) (500 10%) 2.8564 5.08
1m 46s (- 12m 58s) (600 12%) 3.0813 5.36
2m 6s (- 12m 57s) (700 14%) 2.7259 4.99
2m 25s (- 12m 46s) (800 16%) 3.6696 7.56
2m 41s (- 12m 15s) (900 18%) 2.3346 5.30
2m 58s (- 11m 52s) (1000 20%) 2.9128 5.65
3m 14s (- 11m 31s) (1100 22%) 2.2185 5.60
3m 30s (- 11m 6s) (1200 24%) 2.3348 4.35
3m 49s (- 10m 52s) (1300 26%) 3.3978 6.50
4m 6s (- 10m 32s) (1400 28%) 4.0006 7.78
4m 23s (- 10m 14s) (1500 30%) 2.9044 5.31
4m 40s (- 9m 56s) (1600 32%) 2.4708 5.65
4m 59s (- 9m 40s) (1700 34%) 4.0121 8.70
5m 17s (- 9m 24s) (1800 36%) 3.4835 6.28
5m 35s (- 9m 7s) (1900 38%) 2.8825 7.13
5m 50s (- 8m 46s) (2000 40%) 3.4978 6.08
6m 7s (- 8m 27s) (2100 42%) 3.4128 6.53
6m 24s (- 8m 9s) (2200 44%) 2.5112 5.58
6m 43s (- 7m 53s) (2300 46%) 1.9004 4.01
7m 0s (- 7m 35s) (2400 48%) 2.6646 5.49
7m 18s (- 7m 18s) (250

48m 16s (- 18m 18s) (14500 72%) 1.4697 2.54
48m 36s (- 17m 58s) (14600 73%) 0.9834 1.65
48m 53s (- 17m 37s) (14700 73%) 1.4637 2.62
49m 13s (- 17m 17s) (14800 74%) 1.3622 2.32
49m 32s (- 16m 57s) (14900 74%) 2.1577 3.73
49m 46s (- 16m 35s) (15000 75%) 1.8290 3.69
50m 5s (- 16m 15s) (15100 75%) 1.8088 3.56
50m 24s (- 15m 55s) (15200 76%) 1.5181 3.33
50m 45s (- 15m 35s) (15300 76%) 1.3097 3.64
51m 1s (- 15m 14s) (15400 77%) 1.5004 2.25
51m 21s (- 14m 54s) (15500 77%) 1.6789 3.47
51m 42s (- 14m 35s) (15600 78%) 1.8180 3.45
52m 2s (- 14m 15s) (15700 78%) 1.9827 3.57
52m 19s (- 13m 54s) (15800 79%) 1.5858 3.23
52m 41s (- 13m 35s) (15900 79%) 1.4181 2.92
52m 55s (- 13m 13s) (16000 80%) 1.7853 3.14
53m 17s (- 12m 54s) (16100 80%) 1.6849 3.49
53m 37s (- 12m 34s) (16200 81%) 1.5077 3.20
53m 56s (- 12m 14s) (16300 81%) 1.7476 3.03
54m 13s (- 11m 54s) (16400 82%) 1.6106 1.88
54m 30s (- 11m 33s) (16500 82%) 1.0155 2.92
54m 47s (- 11m 13s) (16600 83%) 1.4440 2.11
55m 9s (- 10m 53s) (16700 83%) 1.38

In [31]:
# 10716
torch.save(chatbot.state_dict(), 'C:\\Users\\Jb\\Desktop\\NLP\\saves\\models\\M2DS_2019_final_chatbot_100_75_75_010_5.pth')

In [32]:
chatbot.eval()
z = trainer.ErrorCount(chatbot, dialogues_Master_test_var) # erreurs par réponse

The repartition of errors : [10716, 9, 133, 46, 39, 103, 50, 169, 227, 129, 354, 597, 561, 577, 845, 413, 920, 1365, 271, 501, 185, 299, 32, 76, 319, 97, 545, 44, 4, 7, 1, 5, 0, 6, 4, 15, 1, 3, 0, 0, 45, 1, 0, 0, 1, 1, 0, 1, 0, 0, 22, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 115]


### 6.3.4 Learning rate harved

In [34]:
chatbot.train()
trainer.train(chatbot, dialogues_Master_var_025, n_iters = 5000, learning_rate=0.025, dic = errors)
errors = trainer.train(chatbot, dialogues_Master_var_025, n_iters = 20000, per_dialogue = True, learning_rate=0.025, return_errors = True)

0m 17s (- 14m 36s) (100 2%) 2.9319 6.19
0m 37s (- 14m 59s) (200 4%) 2.9112 5.98
0m 54s (- 14m 13s) (300 6%) 2.6409 4.73
1m 13s (- 14m 2s) (400 8%) 2.4665 4.81
1m 32s (- 13m 51s) (500 10%) 3.2950 7.07
1m 48s (- 13m 15s) (600 12%) 2.4641 4.42
2m 4s (- 12m 46s) (700 14%) 3.7402 5.87
2m 22s (- 12m 27s) (800 16%) 2.7961 6.32
2m 38s (- 12m 2s) (900 18%) 2.4165 4.49
2m 55s (- 11m 40s) (1000 20%) 2.9104 5.26
3m 13s (- 11m 26s) (1100 22%) 3.2430 5.89
3m 33s (- 11m 15s) (1200 24%) 2.5809 4.77
3m 50s (- 10m 56s) (1300 26%) 1.4114 2.66
4m 10s (- 10m 43s) (1400 28%) 2.3757 6.60
4m 25s (- 10m 18s) (1500 30%) 2.7322 4.80
4m 41s (- 9m 57s) (1600 32%) 2.1244 4.47
4m 57s (- 9m 37s) (1700 34%) 2.6114 4.56
5m 14s (- 9m 18s) (1800 36%) 2.4625 5.68
5m 32s (- 9m 2s) (1900 38%) 2.8230 5.29
5m 52s (- 8m 48s) (2000 40%) 2.8078 6.98
6m 7s (- 8m 26s) (2100 42%) 2.1776 3.46
6m 24s (- 8m 9s) (2200 44%) 2.0447 3.70
6m 44s (- 7m 54s) (2300 46%) 2.0962 4.41
7m 2s (- 7m 37s) (2400 48%) 2.0908 4.57
7m 21s (- 7m 21s) (25

48m 52s (- 18m 4s) (14600 73%) 1.0291 2.05
49m 9s (- 17m 43s) (14700 73%) 1.2160 1.93
49m 32s (- 17m 24s) (14800 74%) 1.3119 2.11
49m 51s (- 17m 3s) (14900 74%) 0.9902 2.50
50m 6s (- 16m 42s) (15000 75%) 0.8999 1.36
50m 25s (- 16m 21s) (15100 75%) 1.3647 2.55
50m 45s (- 16m 1s) (15200 76%) 0.9202 1.50
51m 5s (- 15m 41s) (15300 76%) 0.8067 1.72
51m 22s (- 15m 20s) (15400 77%) 1.2818 2.48
51m 39s (- 14m 59s) (15500 77%) 0.5640 1.35
51m 59s (- 14m 39s) (15600 78%) 1.4834 2.05
52m 20s (- 14m 20s) (15700 78%) 1.0644 1.99
52m 40s (- 14m 0s) (15800 79%) 0.7173 1.06
53m 3s (- 13m 40s) (15900 79%) 1.4566 2.49
53m 23s (- 13m 20s) (16000 80%) 1.5707 3.30
53m 40s (- 13m 0s) (16100 80%) 1.2647 2.34
53m 56s (- 12m 39s) (16200 81%) 0.9333 2.38
54m 18s (- 12m 19s) (16300 81%) 0.5553 2.19
54m 34s (- 11m 58s) (16400 82%) 0.8924 2.05
54m 52s (- 11m 38s) (16500 82%) 1.1793 1.55
55m 10s (- 11m 17s) (16600 83%) 1.2322 2.15
55m 28s (- 10m 57s) (16700 83%) 1.0324 1.52
55m 49s (- 10m 38s) (16800 84%) 1.2816 2.

In [35]:
# 12241
torch.save(chatbot.state_dict(), 'C:\\Users\\Jb\\Desktop\\NLP\\saves\\models\\M2DS_2019_final_chatbot_100_75_75_010_6.pth')

In [36]:
chatbot.eval()
z = trainer.ErrorCount(chatbot, dialogues_Master_test_var) # erreurs par réponse

The repartition of errors : [12241, 3, 23, 13, 0, 29, 30, 157, 108, 89, 501, 736, 579, 308, 849, 659, 805, 1282, 123, 442, 156, 204, 42, 5, 91, 118, 81, 45, 25, 40, 0, 0, 0, 0, 1, 15, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 35, 0, 18]


### 6.3.5 Teacher Forcing Ratio = 0.25


In [25]:
chatbot.decoder.tf_ratio = 0.25

In [38]:
chatbot.train()
errors = trainer.train(chatbot, dialogues_Master_var_025, n_iters = 20000, per_dialogue = True, learning_rate=0.025, return_errors = True)

0m 16s (- 55m 2s) (100 0%) 1.3723 2.53
0m 34s (- 56m 25s) (200 1%) 1.0817 3.04
0m 52s (- 57m 12s) (300 1%) 2.0645 5.18
1m 11s (- 58m 33s) (400 2%) 1.5320 2.45
1m 30s (- 58m 40s) (500 2%) 1.1044 1.83
1m 50s (- 59m 17s) (600 3%) 1.6957 3.43
2m 8s (- 59m 7s) (700 3%) 1.4921 2.57
2m 24s (- 57m 53s) (800 4%) 1.9604 2.51
2m 39s (- 56m 31s) (900 4%) 2.1581 3.07
3m 2s (- 57m 43s) (1000 5%) 1.6270 3.05
3m 22s (- 58m 0s) (1100 5%) 1.2497 1.85
3m 44s (- 58m 31s) (1200 6%) 1.1420 1.92
4m 7s (- 59m 23s) (1300 6%) 1.9023 3.47
4m 29s (- 59m 40s) (1400 7%) 1.5787 2.69
4m 47s (- 59m 6s) (1500 7%) 1.1569 1.85
5m 6s (- 58m 48s) (1600 8%) 1.5083 2.72
5m 23s (- 58m 1s) (1700 8%) 1.2049 2.26
5m 43s (- 57m 57s) (1800 9%) 1.2866 1.74
6m 4s (- 57m 56s) (1900 9%) 1.4531 3.33
6m 27s (- 58m 5s) (2000 10%) 1.7634 2.56
6m 52s (- 58m 38s) (2100 10%) 1.9004 3.44
7m 12s (- 58m 18s) (2200 11%) 1.5900 2.95
7m 35s (- 58m 23s) (2300 11%) 1.3595 2.71
7m 56s (- 58m 12s) (2400 12%) 1.6660 2.63
8m 16s (- 57m 54s) (2500 12%) 1

63m 17s (- 2m 17s) (19300 96%) 1.4042 2.31
63m 37s (- 1m 58s) (19400 97%) 1.1557 2.44
63m 57s (- 1m 38s) (19500 97%) 1.9078 3.08
64m 13s (- 1m 18s) (19600 98%) 1.2650 2.07
64m 34s (- 0m 59s) (19700 98%) 1.1655 2.54
64m 51s (- 0m 39s) (19800 99%) 1.2583 2.56
65m 8s (- 0m 19s) (19900 99%) 2.0295 4.20
65m 31s (- 0m 0s) (20000 100%) 1.8516 2.74


In [39]:
torch.save(chatbot.state_dict(), 'C:\\Users\\Jb\\Desktop\\NLP\\saves\\models\\M2DS_2019_final_chatbot_100_75_75_010_7.pth') #26

In [40]:
# 12490
chatbot.eval()
z = trainer.ErrorCount(chatbot, dialogues_Master_test_var) # erreurs par réponse

The repartition of errors : [12490, 8, 75, 0, 0, 22, 46, 192, 114, 142, 651, 590, 415, 402, 833, 337, 762, 1263, 164, 421, 212, 296, 8, 9, 102, 25, 69, 44, 2, 29, 0, 0, 0, 0, 0, 16, 1, 10, 0, 28, 2, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 35, 0, 31]


In [42]:
chatbot.train()
trainer.train(chatbot, dialogues_Master_var_025, n_iters = 5000, learning_rate=0.025, dic = errors)
errors = trainer.train(chatbot, dialogues_Master_var_025, n_iters = 20000, per_dialogue = True, learning_rate=0.025, return_errors = True)

0m 19s (- 15m 40s) (100 2%) 3.5341 7.20
0m 36s (- 14m 46s) (200 4%) 3.9818 8.08
0m 54s (- 14m 7s) (300 6%) 4.5043 9.02
1m 12s (- 13m 57s) (400 8%) 3.9319 8.64
1m 30s (- 13m 33s) (500 10%) 3.9276 8.42
1m 46s (- 13m 0s) (600 12%) 3.8740 7.00
2m 3s (- 12m 40s) (700 14%) 4.2095 7.74
2m 22s (- 12m 25s) (800 16%) 3.8176 9.23
2m 39s (- 12m 4s) (900 18%) 3.6598 7.96
2m 54s (- 11m 37s) (1000 20%) 3.3282 8.39
3m 11s (- 11m 20s) (1100 22%) 3.4094 7.58
3m 29s (- 11m 2s) (1200 24%) 3.2750 7.60
3m 47s (- 10m 46s) (1300 26%) 2.9603 6.02
4m 5s (- 10m 30s) (1400 28%) 3.3116 7.46
4m 22s (- 10m 13s) (1500 30%) 3.2426 7.34
4m 39s (- 9m 53s) (1600 32%) 3.4192 7.29
4m 56s (- 9m 35s) (1700 34%) 4.3189 8.64
5m 13s (- 9m 17s) (1800 36%) 4.0217 8.63
5m 32s (- 9m 1s) (1900 38%) 4.0933 8.59
5m 50s (- 8m 45s) (2000 40%) 4.0266 8.69
6m 7s (- 8m 27s) (2100 42%) 3.6986 8.58
6m 25s (- 8m 10s) (2200 44%) 3.7653 10.37
6m 44s (- 7m 54s) (2300 46%) 3.5436 6.66
7m 2s (- 7m 38s) (2400 48%) 3.7133 8.59
7m 21s (- 7m 21s) (250

47m 52s (- 17m 42s) (14600 73%) 1.8802 3.91
48m 11s (- 17m 22s) (14700 73%) 2.2300 3.50
48m 31s (- 17m 2s) (14800 74%) 1.8626 4.06
48m 51s (- 16m 43s) (14900 74%) 1.3030 2.98
49m 14s (- 16m 24s) (15000 75%) 1.4704 2.85
49m 35s (- 16m 5s) (15100 75%) 1.3355 2.47
49m 54s (- 15m 45s) (15200 76%) 1.3904 2.83
50m 12s (- 15m 25s) (15300 76%) 1.3274 3.54
50m 32s (- 15m 5s) (15400 77%) 1.0980 1.75
50m 50s (- 14m 45s) (15500 77%) 1.0509 2.61
51m 11s (- 14m 26s) (15600 78%) 1.3295 2.57
51m 31s (- 14m 6s) (15700 78%) 1.5614 4.34
51m 51s (- 13m 47s) (15800 79%) 1.3952 2.93
52m 7s (- 13m 26s) (15900 79%) 1.2278 2.64
52m 23s (- 13m 5s) (16000 80%) 1.0074 1.66
52m 43s (- 12m 46s) (16100 80%) 1.4955 2.17
53m 0s (- 12m 26s) (16200 81%) 1.3828 2.73
53m 21s (- 12m 6s) (16300 81%) 1.5133 2.63
53m 41s (- 11m 47s) (16400 82%) 1.2041 2.46
53m 59s (- 11m 27s) (16500 82%) 1.0020 2.50
54m 19s (- 11m 7s) (16600 83%) 1.4379 3.10
54m 35s (- 10m 47s) (16700 83%) 1.5814 2.83
54m 51s (- 10m 26s) (16800 84%) 0.7772 1.

In [43]:
torch.save(chatbot.state_dict(), 'C:\\Users\\Jb\\Desktop\\NLP\\saves\\models\\M2DS_2019_final_chatbot_100_75_75_010_8.pth')

In [44]:
# 11866
chatbot.eval()
z = trainer.ErrorCount(chatbot, dialogues_Master_test_var) # erreurs par réponse

The repartition of errors : [11866, 23, 134, 1, 5, 45, 37, 182, 353, 139, 451, 573, 538, 372, 776, 801, 866, 1184, 206, 542, 183, 225, 1, 20, 80, 53, 33, 47, 10, 50, 0, 0, 0, 0, 1, 16, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 37, 0, 0, 1]


### 6.3.5 Learning rate harved



In [45]:
chatbot.train()
trainer.train(chatbot, dialogues_Master_var_025, n_iters = 5000, learning_rate=0.01, dic = errors)
errors = trainer.train(chatbot, dialogues_Master_var_025, n_iters = 20000, per_dialogue = True, learning_rate=0.01, return_errors = True)

0m 17s (- 14m 38s) (100 2%) 4.9635 11.77
0m 33s (- 13m 29s) (200 4%) 4.6516 9.25
0m 53s (- 13m 53s) (300 6%) 3.4019 7.95
1m 12s (- 13m 56s) (400 8%) 3.2885 6.00
1m 29s (- 13m 29s) (500 10%) 4.5414 8.15
1m 50s (- 13m 30s) (600 12%) 4.0610 8.46
2m 9s (- 13m 16s) (700 14%) 4.1660 11.18
2m 26s (- 12m 49s) (800 16%) 3.4008 8.38
2m 44s (- 12m 30s) (900 18%) 3.6709 9.29
3m 1s (- 12m 4s) (1000 20%) 3.3978 7.01
3m 19s (- 11m 46s) (1100 22%) 3.5064 9.03
3m 38s (- 11m 32s) (1200 24%) 3.2641 7.78
3m 55s (- 11m 9s) (1300 26%) 3.2705 7.08
4m 13s (- 10m 51s) (1400 28%) 3.8254 9.33
4m 31s (- 10m 33s) (1500 30%) 2.9220 6.53
4m 48s (- 10m 13s) (1600 32%) 3.1684 6.28
5m 5s (- 9m 52s) (1700 34%) 3.9478 8.45
5m 23s (- 9m 35s) (1800 36%) 4.4450 9.44
5m 41s (- 9m 16s) (1900 38%) 4.3732 9.35
5m 58s (- 8m 58s) (2000 40%) 3.2506 5.83
6m 19s (- 8m 44s) (2100 42%) 4.1080 8.86
6m 38s (- 8m 26s) (2200 44%) 3.9277 9.41
6m 57s (- 8m 10s) (2300 46%) 3.5515 8.87
7m 16s (- 7m 52s) (2400 48%) 2.6827 7.33
7m 33s (- 7m 33s

48m 37s (- 18m 26s) (14500 72%) 1.0238 1.91
48m 56s (- 18m 6s) (14600 73%) 0.8042 1.43
49m 17s (- 17m 46s) (14700 73%) 0.9816 1.83
49m 39s (- 17m 26s) (14800 74%) 1.0464 2.14
49m 55s (- 17m 5s) (14900 74%) 0.7901 1.28
50m 13s (- 16m 44s) (15000 75%) 1.5336 2.40
50m 28s (- 16m 22s) (15100 75%) 1.3901 2.59
50m 48s (- 16m 2s) (15200 76%) 0.9505 2.28
51m 9s (- 15m 42s) (15300 76%) 1.4202 2.70
51m 30s (- 15m 23s) (15400 77%) 1.7355 2.66
51m 49s (- 15m 2s) (15500 77%) 1.0512 2.39
52m 6s (- 14m 41s) (15600 78%) 0.8835 1.49
52m 24s (- 14m 21s) (15700 78%) 1.0272 2.19
52m 51s (- 14m 3s) (15800 79%) 1.2826 2.97
53m 13s (- 13m 43s) (15900 79%) 1.9275 3.76
53m 31s (- 13m 22s) (16000 80%) 1.2577 2.35
53m 48s (- 13m 1s) (16100 80%) 1.0899 2.28
54m 10s (- 12m 42s) (16200 81%) 1.3782 2.28
54m 30s (- 12m 22s) (16300 81%) 1.0990 2.05
54m 49s (- 12m 2s) (16400 82%) 1.3870 2.52
55m 6s (- 11m 41s) (16500 82%) 1.1696 2.48
55m 27s (- 11m 21s) (16600 83%) 0.8866 2.37
55m 47s (- 11m 1s) (16700 83%) 1.0523 2.42

In [46]:
torch.save(chatbot.state_dict(), 'C:\\Users\\Jb\\Desktop\\NLP\\saves\\models\\M2DS_2019_final_chatbot_100_75_75_010_9.pth') #27

In [47]:
# 12433
chatbot.eval()
z = trainer.ErrorCount(chatbot, dialogues_Master_test_var) # erreurs par réponse

The repartition of errors : [12433, 0, 109, 5, 6, 33, 1, 179, 182, 156, 615, 666, 356, 701, 776, 563, 625, 1062, 145, 416, 182, 297, 5, 15, 70, 48, 49, 44, 0, 36, 0, 0, 0, 0, 0, 16, 1, 16, 9, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 35]


In [48]:
chatbot.train()
trainer.train(chatbot, dialogues_Master_var_025, n_iters = 5000, learning_rate=0.01, dic = errors)
errors = trainer.train(chatbot, dialogues_Master_var_025, n_iters = 20000, per_dialogue = True, learning_rate=0.01, return_errors = True)

0m 23s (- 18m 59s) (100 2%) 3.9227 8.31
0m 49s (- 19m 37s) (200 4%) 4.3304 8.09
1m 13s (- 19m 19s) (300 6%) 4.5321 8.93
1m 35s (- 18m 12s) (400 8%) 3.3350 6.83
1m 57s (- 17m 33s) (500 10%) 3.7559 8.83
2m 18s (- 16m 53s) (600 12%) 3.5083 9.45
2m 37s (- 16m 9s) (700 14%) 3.9413 9.46
2m 58s (- 15m 36s) (800 16%) 3.1590 6.66
3m 19s (- 15m 6s) (900 18%) 3.9396 8.61
3m 37s (- 14m 30s) (1000 20%) 3.5997 7.72
3m 56s (- 13m 57s) (1100 22%) 4.1736 8.44
4m 16s (- 13m 30s) (1200 24%) 3.7382 8.92
4m 50s (- 13m 47s) (1300 26%) 3.3236 9.57
5m 9s (- 13m 16s) (1400 28%) 3.0332 7.78
5m 28s (- 12m 47s) (1500 30%) 4.5459 11.94
5m 46s (- 12m 16s) (1600 32%) 3.6818 6.74
6m 3s (- 11m 46s) (1700 34%) 3.3047 7.67
6m 21s (- 11m 18s) (1800 36%) 3.2905 6.90
6m 39s (- 10m 51s) (1900 38%) 3.8535 7.88
6m 57s (- 10m 26s) (2000 40%) 3.1903 7.00
7m 18s (- 10m 5s) (2100 42%) 4.0405 9.98
7m 37s (- 9m 42s) (2200 44%) 3.2907 7.04
7m 55s (- 9m 18s) (2300 46%) 3.5664 7.32
8m 13s (- 8m 55s) (2400 48%) 3.4960 9.45
8m 31s (- 8m

49m 19s (- 18m 14s) (14600 73%) 0.5508 1.13
49m 42s (- 17m 55s) (14700 73%) 1.0298 1.61
50m 1s (- 17m 34s) (14800 74%) 1.3393 3.69
50m 21s (- 17m 14s) (14900 74%) 1.0106 2.67
50m 38s (- 16m 52s) (15000 75%) 1.3553 3.29
51m 0s (- 16m 33s) (15100 75%) 0.7322 1.94
51m 20s (- 16m 12s) (15200 76%) 1.5995 3.05
51m 42s (- 15m 52s) (15300 76%) 1.2474 2.26
52m 3s (- 15m 32s) (15400 77%) 1.4138 3.55
52m 22s (- 15m 12s) (15500 77%) 1.5495 3.70
52m 41s (- 14m 51s) (15600 78%) 1.1762 1.85
53m 5s (- 14m 32s) (15700 78%) 1.1606 2.47
53m 22s (- 14m 11s) (15800 79%) 1.0514 2.22
53m 41s (- 13m 50s) (15900 79%) 1.0865 2.68
54m 1s (- 13m 30s) (16000 80%) 0.7125 1.81
54m 24s (- 13m 10s) (16100 80%) 0.9044 2.08
54m 41s (- 12m 49s) (16200 81%) 0.6949 1.76
55m 5s (- 12m 30s) (16300 81%) 0.6935 1.62
55m 29s (- 12m 10s) (16400 82%) 1.1071 2.30
55m 49s (- 11m 50s) (16500 82%) 1.1538 2.01
56m 10s (- 11m 30s) (16600 83%) 1.2250 2.13
56m 28s (- 11m 9s) (16700 83%) 0.4169 0.93
56m 51s (- 10m 49s) (16800 84%) 0.8805 

In [49]:
torch.save(chatbot.state_dict(), 'C:\\Users\\Jb\\Desktop\\NLP\\saves\\models\\M2DS_2019_final_chatbot_100_75_75_010_10.pth') #27

In [50]:
# 12488
chatbot.eval()
z = trainer.ErrorCount(chatbot, dialogues_Master_test_var) # erreurs par réponse

The repartition of errors : [12488, 0, 86, 0, 0, 67, 13, 278, 224, 101, 508, 441, 319, 553, 1011, 564, 770, 1179, 116, 381, 181, 255, 14, 6, 80, 49, 46, 40, 4, 3, 7, 1, 0, 0, 6, 16, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 35]


### 6.3.6 Learning rate harved

In [56]:
chatbot.train()
trainer.train(chatbot, dialogues_Master_var_025, n_iters = 10000, learning_rate=0.005, dic = errors)
errors = trainer.train(chatbot, dialogues_Master_var_025, n_iters = 20000, per_dialogue = True, learning_rate=0.005, return_errors = True)

0m 17s (- 28m 44s) (100 1%) 4.1081 10.04
0m 35s (- 28m 54s) (200 2%) 4.3142 10.11
0m 53s (- 29m 2s) (300 3%) 3.5662 7.99
1m 11s (- 28m 32s) (400 4%) 4.9158 11.25
1m 28s (- 28m 6s) (500 5%) 3.7956 9.27
1m 46s (- 27m 42s) (600 6%) 3.9526 8.32
2m 4s (- 27m 37s) (700 7%) 3.8127 9.26
2m 23s (- 27m 27s) (800 8%) 4.5837 9.28
2m 41s (- 27m 8s) (900 9%) 3.8847 10.03
2m 59s (- 26m 52s) (1000 10%) 4.0548 11.49
3m 16s (- 26m 26s) (1100 11%) 3.5826 8.55
3m 31s (- 25m 53s) (1200 12%) 3.2986 7.99
3m 48s (- 25m 28s) (1300 13%) 3.5051 8.04
4m 5s (- 25m 11s) (1400 14%) 3.7115 9.81
4m 24s (- 24m 58s) (1500 15%) 3.7888 10.51
4m 41s (- 24m 35s) (1600 16%) 3.1015 8.32
4m 58s (- 24m 18s) (1700 17%) 4.7608 11.51
5m 17s (- 24m 7s) (1800 18%) 3.2803 8.62
5m 36s (- 23m 55s) (1900 19%) 4.0212 10.79
5m 53s (- 23m 33s) (2000 20%) 3.7428 7.22
6m 11s (- 23m 18s) (2100 21%) 3.3995 8.31
6m 28s (- 22m 58s) (2200 22%) 3.3098 8.19
6m 46s (- 22m 41s) (2300 23%) 4.0138 8.96
7m 5s (- 22m 26s) (2400 24%) 2.5665 5.81
7m 22s (-

32m 35s (- 34m 36s) (9700 48%) 0.5796 1.38
32m 53s (- 34m 14s) (9800 49%) 1.0160 2.36
33m 10s (- 33m 50s) (9900 49%) 1.2048 2.38
33m 31s (- 33m 31s) (10000 50%) 1.1388 2.78
33m 51s (- 33m 11s) (10100 50%) 0.8908 1.65
34m 15s (- 32m 55s) (10200 51%) 1.2541 2.59
34m 37s (- 32m 36s) (10300 51%) 1.6588 2.82
34m 56s (- 32m 15s) (10400 52%) 1.2138 2.44
35m 19s (- 31m 57s) (10500 52%) 0.9814 1.58
35m 38s (- 31m 36s) (10600 53%) 1.1315 2.28
36m 4s (- 31m 20s) (10700 53%) 1.0447 2.31
36m 26s (- 31m 2s) (10800 54%) 1.1321 2.22
36m 45s (- 30m 41s) (10900 54%) 0.7499 1.64
37m 7s (- 30m 22s) (11000 55%) 1.1087 3.07
37m 28s (- 30m 2s) (11100 55%) 0.8691 1.68
37m 49s (- 29m 43s) (11200 56%) 1.1023 2.31
38m 11s (- 29m 24s) (11300 56%) 0.6224 1.00
38m 34s (- 29m 5s) (11400 56%) 0.7818 2.05
38m 51s (- 28m 43s) (11500 57%) 1.3024 2.45
39m 12s (- 28m 23s) (11600 57%) 1.3846 2.52
39m 28s (- 28m 0s) (11700 58%) 0.7185 1.51
39m 43s (- 27m 36s) (11800 59%) 0.7987 1.58
40m 0s (- 27m 13s) (11900 59%) 1.0561 2.4

In [57]:
torch.save(chatbot.state_dict(), 'C:\\Users\\Jb\\Desktop\\NLP\\saves\\models\\M2DS_2019_final_chatbot_100_75_75_010_11.pth') #27

In [58]:
# 12588
chatbot.eval()
z = trainer.ErrorCount(chatbot, dialogues_Master_test_var) # erreurs par réponse

The repartition of errors : [12588, 1, 101, 0, 0, 78, 2, 154, 208, 114, 508, 438, 225, 454, 1088, 651, 824, 1168, 110, 373, 179, 218, 2, 45, 45, 45, 97, 44, 6, 5, 0, 1, 0, 6, 2, 16, 0, 16, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 35]


In [60]:
chatbot.train()
trainer.train(chatbot, dialogues_Master_var_025, n_iters = 60000, learning_rate=0.005, dic = errors)
errors = trainer.train(chatbot, dialogues_Master_var_025, n_iters = 20000, per_dialogue = True, learning_rate=0.005, return_errors = True)

0m 19s (- 196m 30s) (100 0%) 3.2465 8.28
0m 35s (- 177m 16s) (200 0%) 3.6700 9.16
0m 55s (- 184m 40s) (300 0%) 2.6343 6.06
1m 17s (- 191m 38s) (400 0%) 4.1854 10.14
1m 37s (- 193m 40s) (500 0%) 3.7368 8.14
1m 54s (- 189m 13s) (600 1%) 3.4720 7.41
2m 12s (- 187m 26s) (700 1%) 3.1721 6.91
2m 29s (- 184m 26s) (800 1%) 3.3086 7.69
2m 48s (- 184m 39s) (900 1%) 3.1879 8.16
3m 7s (- 184m 19s) (1000 1%) 4.4252 10.70
3m 26s (- 184m 11s) (1100 1%) 3.3564 9.57
3m 45s (- 184m 1s) (1200 2%) 3.4902 7.74
4m 4s (- 184m 6s) (1300 2%) 3.4958 11.19
4m 23s (- 183m 33s) (1400 2%) 2.8419 6.70
4m 40s (- 182m 23s) (1500 2%) 4.0916 10.19
4m 59s (- 182m 3s) (1600 2%) 3.8620 10.83
5m 19s (- 182m 28s) (1700 2%) 4.0397 12.53
5m 37s (- 181m 51s) (1800 3%) 3.7597 7.59
5m 57s (- 182m 9s) (1900 3%) 3.9038 9.64
6m 13s (- 180m 25s) (2000 3%) 3.7508 9.37
6m 31s (- 179m 56s) (2100 3%) 2.5869 7.51
6m 51s (- 180m 9s) (2200 3%) 3.0138 6.87
7m 11s (- 180m 19s) (2300 3%) 3.0433 6.68
7m 30s (- 180m 11s) (2400 4%) 3.3544 9.01
7m

57m 47s (- 125m 39s) (18900 31%) 1.8159 5.28
58m 2s (- 125m 15s) (19000 31%) 2.5188 5.58
58m 22s (- 124m 59s) (19100 31%) 2.6470 8.26
58m 41s (- 124m 42s) (19200 32%) 2.5647 8.68
58m 59s (- 124m 23s) (19300 32%) 2.8518 7.25
59m 15s (- 124m 1s) (19400 32%) 2.1947 5.28
59m 34s (- 123m 43s) (19500 32%) 2.0553 5.25
59m 50s (- 123m 21s) (19600 32%) 2.4046 7.97
60m 7s (- 123m 0s) (19700 32%) 2.6292 7.02
60m 25s (- 122m 41s) (19800 33%) 2.8430 5.46
60m 45s (- 122m 26s) (19900 33%) 2.2286 5.24
61m 2s (- 122m 5s) (20000 33%) 3.4066 10.24
61m 22s (- 121m 49s) (20100 33%) 2.5339 7.92
61m 40s (- 121m 31s) (20200 33%) 3.0271 7.70
61m 58s (- 121m 13s) (20300 33%) 2.3308 6.64
62m 18s (- 120m 57s) (20400 34%) 2.7826 6.29
62m 34s (- 120m 34s) (20500 34%) 2.4879 6.56
62m 54s (- 120m 18s) (20600 34%) 2.3209 7.21
63m 10s (- 119m 56s) (20700 34%) 2.4564 6.37
63m 29s (- 119m 39s) (20800 34%) 2.6680 5.57
63m 47s (- 119m 19s) (20900 34%) 2.4635 6.03
64m 3s (- 118m 57s) (21000 35%) 2.2580 6.19
64m 22s (- 118m 

113m 26s (- 68m 32s) (37400 62%) 2.0825 5.09
113m 45s (- 68m 15s) (37500 62%) 1.8345 6.00
114m 1s (- 67m 55s) (37600 62%) 3.3549 7.78
114m 18s (- 67m 37s) (37700 62%) 2.1296 4.78
114m 37s (- 67m 19s) (37800 63%) 1.5870 4.38
114m 53s (- 66m 59s) (37900 63%) 2.2613 5.42
115m 11s (- 66m 41s) (38000 63%) 2.1474 6.84
115m 28s (- 66m 22s) (38100 63%) 2.3046 5.23
115m 48s (- 66m 5s) (38200 63%) 2.3227 5.90
116m 5s (- 65m 46s) (38300 63%) 2.4557 7.23
116m 23s (- 65m 28s) (38400 64%) 2.5697 5.43
116m 41s (- 65m 10s) (38500 64%) 1.9541 5.39
116m 59s (- 64m 51s) (38600 64%) 2.1135 6.09
117m 17s (- 64m 33s) (38700 64%) 2.5757 7.40
117m 35s (- 64m 15s) (38800 64%) 2.0895 5.57
117m 54s (- 63m 57s) (38900 64%) 1.8336 4.31
118m 12s (- 63m 38s) (39000 65%) 2.4317 4.97
118m 30s (- 63m 20s) (39100 65%) 2.2248 5.43
118m 48s (- 63m 2s) (39200 65%) 2.6252 6.86
119m 5s (- 62m 43s) (39300 65%) 1.6328 4.40
119m 24s (- 62m 25s) (39400 65%) 2.5087 5.17
119m 42s (- 62m 7s) (39500 65%) 2.4085 5.53
120m 0s (- 61m 4

169m 4s (- 12m 43s) (55800 93%) 1.6594 4.08
169m 21s (- 12m 25s) (55900 93%) 2.1740 4.80
169m 39s (- 12m 7s) (56000 93%) 1.7467 4.82
169m 59s (- 11m 49s) (56100 93%) 1.8708 5.21
170m 16s (- 11m 30s) (56200 93%) 1.4217 4.15
170m 35s (- 11m 12s) (56300 93%) 2.0943 6.25
170m 53s (- 10m 54s) (56400 94%) 1.5610 4.59
171m 11s (- 10m 36s) (56500 94%) 1.5239 3.33
171m 29s (- 10m 18s) (56600 94%) 2.2584 6.72
171m 48s (- 9m 59s) (56700 94%) 1.9605 4.83
172m 7s (- 9m 41s) (56800 94%) 2.2009 5.41
172m 24s (- 9m 23s) (56900 94%) 2.1050 4.31
172m 44s (- 9m 5s) (57000 95%) 1.8211 5.12
173m 3s (- 8m 47s) (57100 95%) 1.9894 6.24
173m 22s (- 8m 29s) (57200 95%) 1.7646 4.41
173m 40s (- 8m 11s) (57300 95%) 1.6607 5.02
173m 59s (- 7m 52s) (57400 95%) 2.0443 6.05
174m 18s (- 7m 34s) (57500 95%) 1.4360 2.88
174m 37s (- 7m 16s) (57600 96%) 1.5498 3.57
174m 57s (- 6m 58s) (57700 96%) 1.7897 5.25
175m 14s (- 6m 40s) (57800 96%) 1.5130 4.32
175m 30s (- 6m 21s) (57900 96%) 1.8828 4.48
175m 50s (- 6m 3s) (58000 96

50m 38s (- 17m 20s) (14900 74%) 1.5507 3.30
50m 55s (- 16m 58s) (15000 75%) 1.0885 2.68
51m 10s (- 16m 36s) (15100 75%) 1.0436 2.13
51m 34s (- 16m 17s) (15200 76%) 0.7838 1.67
51m 57s (- 15m 57s) (15300 76%) 0.7125 1.43
52m 18s (- 15m 37s) (15400 77%) 1.0831 1.97
52m 39s (- 15m 17s) (15500 77%) 1.0610 2.45
53m 4s (- 14m 58s) (15600 78%) 1.2299 2.64
53m 27s (- 14m 38s) (15700 78%) 1.1982 2.36
53m 47s (- 14m 17s) (15800 79%) 1.2803 1.88
54m 13s (- 13m 58s) (15900 79%) 1.4293 2.17
54m 33s (- 13m 38s) (16000 80%) 0.5866 0.95
54m 55s (- 13m 18s) (16100 80%) 0.9325 1.69
55m 12s (- 12m 57s) (16200 81%) 1.0500 2.35
55m 34s (- 12m 36s) (16300 81%) 0.7932 1.57
55m 51s (- 12m 15s) (16400 82%) 0.5921 1.30
56m 15s (- 11m 55s) (16500 82%) 1.6476 3.60
56m 38s (- 11m 36s) (16600 83%) 1.1956 2.42
56m 58s (- 11m 15s) (16700 83%) 0.7531 1.87
57m 18s (- 10m 54s) (16800 84%) 0.9200 3.39
57m 38s (- 10m 34s) (16900 84%) 0.8249 1.69
58m 1s (- 10m 14s) (17000 85%) 1.0593 1.95
58m 21s (- 9m 53s) (17100 85%) 0.9

In [61]:
torch.save(chatbot.state_dict(), 'C:\\Users\\Jb\\Desktop\\NLP\\saves\\models\\M2DS_2019_final_chatbot_100_75_75_010_12.pth') #27

In [62]:
# 12374
chatbot.eval()
z = trainer.ErrorCount(chatbot, dialogues_Master_test_var) # erreurs par réponse

The repartition of errors : [12374, 2, 60, 1, 0, 62, 16, 83, 83, 118, 618, 507, 337, 444, 1128, 840, 672, 1194, 144, 307, 199, 262, 0, 12, 99, 57, 101, 44, 0, 21, 0, 0, 0, 0, 0, 16, 0, 3, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 35, 0, 2]


In [63]:
chatbot.train()
trainer.train(chatbot, dialogues_Master_var_025, n_iters = 10000, learning_rate=0.005, dic = errors)
errors = trainer.train(chatbot, dialogues_Master_var_025, n_iters = 20000, per_dialogue = True, learning_rate=0.005, return_errors = True)

0m 16s (- 27m 38s) (100 1%) 3.6135 7.25
0m 34s (- 28m 30s) (200 2%) 3.7762 9.02
0m 53s (- 28m 39s) (300 3%) 3.0814 7.65
1m 12s (- 28m 50s) (400 4%) 3.7223 7.75
1m 30s (- 28m 41s) (500 5%) 4.0737 9.70
1m 47s (- 28m 11s) (600 6%) 3.2562 7.51
2m 6s (- 28m 6s) (700 7%) 3.8689 8.73
2m 24s (- 27m 41s) (800 8%) 3.3749 7.05
2m 40s (- 26m 59s) (900 9%) 3.6958 7.72
2m 58s (- 26m 47s) (1000 10%) 4.0948 9.11
3m 15s (- 26m 17s) (1100 11%) 3.6549 7.90
3m 33s (- 26m 2s) (1200 12%) 3.6041 7.79
3m 52s (- 25m 58s) (1300 13%) 3.1647 8.31
4m 10s (- 25m 39s) (1400 14%) 3.8056 9.65
4m 25s (- 25m 7s) (1500 15%) 3.9755 8.16
4m 44s (- 24m 52s) (1600 16%) 3.0236 6.66
5m 0s (- 24m 28s) (1700 17%) 3.3719 7.14
5m 18s (- 24m 9s) (1800 18%) 3.1513 7.34
5m 36s (- 23m 52s) (1900 19%) 3.8321 7.81
5m 53s (- 23m 34s) (2000 20%) 3.7516 8.65
6m 10s (- 23m 14s) (2100 21%) 3.6299 7.65
6m 28s (- 22m 58s) (2200 22%) 3.0516 6.41
6m 47s (- 22m 43s) (2300 23%) 2.9185 8.04
7m 4s (- 22m 24s) (2400 24%) 3.1975 6.53
7m 23s (- 22m 11s

32m 39s (- 34m 40s) (9700 48%) 0.8415 1.53
33m 3s (- 34m 24s) (9800 49%) 0.5341 1.24
33m 23s (- 34m 3s) (9900 49%) 1.1535 1.95
33m 44s (- 33m 44s) (10000 50%) 1.5974 4.68
34m 9s (- 33m 28s) (10100 50%) 0.7698 2.47
34m 30s (- 33m 9s) (10200 51%) 1.3782 2.82
34m 48s (- 32m 46s) (10300 51%) 1.0570 2.32
35m 7s (- 32m 25s) (10400 52%) 0.5897 1.56
35m 28s (- 32m 5s) (10500 52%) 0.6334 1.20
35m 49s (- 31m 46s) (10600 53%) 1.0119 1.97
36m 10s (- 31m 26s) (10700 53%) 1.2668 1.89
36m 27s (- 31m 3s) (10800 54%) 0.5639 1.09
36m 45s (- 30m 41s) (10900 54%) 0.5847 1.41
37m 4s (- 30m 19s) (11000 55%) 1.2074 2.25
37m 21s (- 29m 57s) (11100 55%) 0.5987 2.06
37m 40s (- 29m 36s) (11200 56%) 1.0019 1.76
37m 58s (- 29m 14s) (11300 56%) 0.6920 1.56
38m 19s (- 28m 54s) (11400 56%) 1.1324 2.08
38m 41s (- 28m 35s) (11500 57%) 1.1219 2.63
38m 59s (- 28m 14s) (11600 57%) 0.9752 1.50
39m 20s (- 27m 54s) (11700 58%) 1.3058 2.69
39m 41s (- 27m 35s) (11800 59%) 1.7079 2.91
40m 4s (- 27m 16s) (11900 59%) 0.6173 0.88


In [64]:
torch.save(chatbot.state_dict(), 'C:\\Users\\Jb\\Desktop\\NLP\\saves\\models\\M2DS_2019_final_chatbot_100_75_75_010_13.pth')

In [65]:
# 12577
chatbot.eval()
z = trainer.ErrorCount(chatbot, dialogues_Master_test_var) # erreurs par réponse

The repartition of errors : [12577, 12, 53, 6, 0, 31, 1, 86, 162, 77, 674, 528, 340, 404, 1229, 667, 603, 1068, 127, 367, 171, 277, 0, 30, 79, 104, 39, 44, 0, 28, 0, 0, 0, 0, 0, 16, 0, 11, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 35, 0, 3]


In [68]:
chatbot.train()
trainer.train(chatbot, dialogues_Master_var, n_iters = 5000, learning_rate=0.005, dic = errors)
errors = trainer.train(chatbot, dialogues_Master_var, n_iters = 10000, per_dialogue = True, learning_rate=0.005, return_errors = True)

0m 56s (- 45m 45s) (100 2%) 0.3428 0.69
1m 28s (- 35m 33s) (200 4%) 0.5898 1.39
1m 54s (- 30m 0s) (300 6%) 0.3612 1.48
2m 20s (- 26m 50s) (400 8%) 0.2967 2.34
2m 42s (- 24m 25s) (500 10%) 0.7220 1.96
3m 7s (- 22m 54s) (600 12%) 0.3097 0.99
3m 27s (- 21m 16s) (700 14%) 0.4622 2.07
3m 51s (- 20m 15s) (800 16%) 0.3493 1.09
4m 13s (- 19m 15s) (900 18%) 0.5096 2.10
4m 33s (- 18m 13s) (1000 20%) 0.3640 1.14
4m 54s (- 17m 23s) (1100 22%) 0.1220 0.21
5m 15s (- 16m 40s) (1200 24%) 0.0542 0.11
5m 38s (- 16m 3s) (1300 26%) 0.6208 3.00
5m 59s (- 15m 23s) (1400 28%) 0.4347 0.73
6m 20s (- 14m 48s) (1500 30%) 0.4390 1.75
6m 40s (- 14m 11s) (1600 32%) 0.3487 1.66
7m 1s (- 13m 38s) (1700 34%) 0.2798 0.99
7m 21s (- 13m 5s) (1800 36%) 0.3848 1.01
7m 40s (- 12m 31s) (1900 38%) 0.2013 0.46
8m 0s (- 12m 0s) (2000 40%) 0.5299 1.88
8m 19s (- 11m 30s) (2100 42%) 0.4968 1.40
8m 38s (- 11m 0s) (2200 44%) 0.3173 0.60
8m 58s (- 10m 32s) (2300 46%) 0.1688 0.85
9m 20s (- 10m 6s) (2400 48%) 0.6037 2.12
9m 38s (- 9m 3

In [69]:
torch.save(chatbot.state_dict(), 'C:\\Users\\Jb\\Desktop\\NLP\\saves\\models\\M2DS_2019_final_chatbot_100_75_75_010_14.pth')

In [70]:
# 12329
chatbot.eval()
z = trainer.ErrorCount(chatbot, dialogues_Master_test_var) # erreurs par réponse

The repartition of errors : [12329, 14, 34, 0, 0, 32, 1, 101, 161, 51, 624, 475, 320, 377, 1240, 740, 654, 1077, 111, 352, 158, 288, 2, 44, 78, 100, 37, 45, 111, 181, 1, 0, 0, 0, 0, 16, 0, 4, 0, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 35, 0, 50]


In [71]:
chatbot.train()
trainer.train(chatbot, dialogues_Master_var_025, n_iters = 10000, learning_rate=0.005, dic = errors)
errors = trainer.train(chatbot, dialogues_Master_var_025, n_iters = 30000, per_dialogue = True, learning_rate=0.005, return_errors = True)

0m 27s (- 45m 33s) (100 1%) 4.2292 13.00
0m 52s (- 43m 7s) (200 2%) 4.0538 14.09
1m 14s (- 40m 10s) (300 3%) 2.9150 11.46
1m 34s (- 37m 39s) (400 4%) 3.1665 11.39
1m 54s (- 36m 14s) (500 5%) 3.0809 10.94
2m 14s (- 35m 11s) (600 6%) 2.8251 8.08
2m 34s (- 34m 17s) (700 7%) 2.3905 10.24
2m 57s (- 33m 56s) (800 8%) 2.7185 12.12
3m 18s (- 33m 23s) (900 9%) 4.1725 18.69
3m 38s (- 32m 45s) (1000 10%) 3.5401 9.83
3m 59s (- 32m 17s) (1100 11%) 2.5911 11.69
4m 20s (- 31m 50s) (1200 12%) 2.5333 9.92
4m 39s (- 31m 12s) (1300 13%) 3.6074 12.55
5m 0s (- 30m 45s) (1400 14%) 1.7944 9.46
5m 23s (- 30m 30s) (1500 15%) 2.1894 9.83
5m 45s (- 30m 14s) (1600 16%) 2.1400 9.35
6m 5s (- 29m 45s) (1700 17%) 2.3594 8.31
6m 25s (- 29m 17s) (1800 18%) 2.5402 12.22
6m 45s (- 28m 46s) (1900 19%) 2.3450 10.02
7m 8s (- 28m 32s) (2000 20%) 2.2929 10.12
7m 29s (- 28m 12s) (2100 21%) 2.3322 10.70
7m 48s (- 27m 40s) (2200 22%) 2.3084 10.88
8m 9s (- 27m 17s) (2300 23%) 2.0536 11.14
8m 29s (- 26m 54s) (2400 24%) 1.4237 10.6

32m 32s (- 69m 9s) (9600 32%) 1.2016 2.84
32m 47s (- 68m 37s) (9700 32%) 1.4215 3.60
33m 8s (- 68m 19s) (9800 32%) 0.8545 1.61
33m 29s (- 68m 0s) (9900 33%) 0.7641 1.35
33m 53s (- 67m 46s) (10000 33%) 0.7740 1.23
34m 17s (- 67m 34s) (10100 33%) 0.9679 1.85
34m 39s (- 67m 17s) (10200 34%) 0.7390 1.53
34m 55s (- 66m 48s) (10300 34%) 1.3876 1.93
35m 12s (- 66m 20s) (10400 34%) 0.7738 1.24
35m 31s (- 65m 59s) (10500 35%) 0.7970 1.38
35m 55s (- 65m 45s) (10600 35%) 1.1886 2.22
36m 17s (- 65m 26s) (10700 35%) 1.6938 2.98
36m 37s (- 65m 6s) (10800 36%) 1.1809 2.04
36m 58s (- 64m 47s) (10900 36%) 1.0357 1.65
37m 19s (- 64m 28s) (11000 36%) 1.6597 2.57
37m 39s (- 64m 7s) (11100 37%) 1.1183 2.25
37m 58s (- 63m 45s) (11200 37%) 1.4106 2.85
38m 19s (- 63m 25s) (11300 37%) 0.7997 1.49
38m 36s (- 62m 58s) (11400 38%) 1.6025 3.05
38m 54s (- 62m 35s) (11500 38%) 0.5358 1.11
39m 14s (- 62m 14s) (11600 38%) 0.8665 1.47
39m 35s (- 61m 55s) (11700 39%) 1.8964 4.32
40m 0s (- 61m 42s) (11800 39%) 0.7183 1.3

97m 0s (- 5m 27s) (28400 94%) 0.6342 1.26
97m 22s (- 5m 7s) (28500 95%) 0.9471 2.09
97m 43s (- 4m 47s) (28600 95%) 0.6731 1.80
98m 2s (- 4m 26s) (28700 95%) 0.5305 0.73
98m 20s (- 4m 5s) (28800 96%) 0.7645 1.31
98m 39s (- 3m 45s) (28900 96%) 0.5930 0.91
99m 0s (- 3m 24s) (29000 96%) 1.6149 2.64
99m 22s (- 3m 4s) (29100 97%) 1.5682 3.29
99m 44s (- 2m 43s) (29200 97%) 1.0822 1.67
100m 4s (- 2m 23s) (29300 97%) 0.6870 1.44
100m 25s (- 2m 2s) (29400 98%) 0.9294 2.26
100m 46s (- 1m 42s) (29500 98%) 0.7529 1.59
101m 8s (- 1m 22s) (29600 98%) 0.7610 1.79
101m 31s (- 1m 1s) (29700 99%) 1.1854 3.14
101m 52s (- 0m 41s) (29800 99%) 0.6464 1.08
102m 14s (- 0m 20s) (29900 99%) 1.5295 3.44
102m 33s (- 0m 0s) (30000 100%) 1.1718 1.67


In [72]:
torch.save(chatbot.state_dict(), 'C:\\Users\\Jb\\Desktop\\NLP\\saves\\models\\M2DS_2019_final_chatbot_100_75_75_010_15.pth')

In [73]:
# 12494 
chatbot.eval()
z = trainer.ErrorCount(chatbot, dialogues_Master_test_var) # erreurs par réponse

The repartition of errors : [12494, 0, 39, 0, 0, 42, 4, 102, 71, 92, 883, 492, 310, 416, 989, 791, 599, 1101, 171, 297, 177, 299, 2, 23, 85, 152, 58, 45, 9, 17, 0, 0, 0, 0, 0, 16, 0, 16, 0, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 35, 0, 3]


In [74]:
chatbot.train()
trainer.train(chatbot, dialogues_Master_var_025, n_iters = 10000, learning_rate=0.005, dic = errors)
errors = trainer.train(chatbot, dialogues_Master_var_025, n_iters = 30000, per_dialogue = True, learning_rate=0.005, return_errors = True)

0m 20s (- 33m 47s) (100 1%) 3.9516 8.15
0m 39s (- 32m 23s) (200 2%) 3.9862 9.11
0m 57s (- 30m 48s) (300 3%) 3.9999 8.33
1m 16s (- 30m 34s) (400 4%) 4.2006 8.50
1m 34s (- 29m 55s) (500 5%) 3.8884 6.85
1m 51s (- 29m 7s) (600 6%) 3.4811 7.41
2m 10s (- 28m 52s) (700 7%) 3.9843 8.72
2m 30s (- 28m 52s) (800 8%) 3.6862 9.76
2m 48s (- 28m 21s) (900 9%) 2.9953 7.14
3m 6s (- 28m 1s) (1000 10%) 3.8648 11.00
3m 24s (- 27m 38s) (1100 11%) 3.1290 7.46
3m 42s (- 27m 15s) (1200 12%) 3.2586 8.62
4m 0s (- 26m 52s) (1300 13%) 3.3029 7.04
4m 20s (- 26m 38s) (1400 14%) 3.0450 7.70
4m 38s (- 26m 16s) (1500 15%) 3.2254 8.33
4m 55s (- 25m 51s) (1600 16%) 3.1012 7.39
5m 14s (- 25m 37s) (1700 17%) 3.6690 9.29
5m 33s (- 25m 19s) (1800 18%) 3.3976 9.13
5m 52s (- 25m 4s) (1900 19%) 3.7785 9.28
6m 10s (- 24m 43s) (2000 20%) 3.7651 7.90
6m 27s (- 24m 19s) (2100 21%) 3.8414 7.29
6m 45s (- 23m 56s) (2200 22%) 2.6944 5.45
7m 3s (- 23m 36s) (2300 23%) 3.6500 8.24
7m 21s (- 23m 19s) (2400 24%) 3.8826 9.61
7m 39s (- 22m 5

32m 34s (- 69m 13s) (9600 32%) 0.3455 0.79
32m 51s (- 68m 46s) (9700 32%) 0.6519 1.25
33m 11s (- 68m 25s) (9800 32%) 1.2456 2.01
33m 28s (- 67m 57s) (9900 33%) 1.0050 1.69
33m 50s (- 67m 40s) (10000 33%) 0.8547 1.45
34m 10s (- 67m 20s) (10100 33%) 0.9780 2.78
34m 29s (- 66m 57s) (10200 34%) 0.9769 1.79
34m 49s (- 66m 36s) (10300 34%) 1.4501 3.60
35m 11s (- 66m 18s) (10400 34%) 0.7219 1.73
35m 29s (- 65m 54s) (10500 35%) 0.8595 2.60
35m 52s (- 65m 39s) (10600 35%) 0.6642 1.35
36m 15s (- 65m 24s) (10700 35%) 0.4383 0.70
36m 35s (- 65m 2s) (10800 36%) 0.9150 2.05
36m 54s (- 64m 40s) (10900 36%) 0.3780 1.59
37m 12s (- 64m 16s) (11000 36%) 0.7032 1.39
37m 36s (- 64m 2s) (11100 37%) 0.7185 1.37
37m 56s (- 63m 41s) (11200 37%) 0.5067 1.01
38m 16s (- 63m 20s) (11300 37%) 1.2683 1.58
38m 34s (- 62m 57s) (11400 38%) 1.0465 2.37
38m 54s (- 62m 36s) (11500 38%) 0.9905 1.73
39m 17s (- 62m 19s) (11600 38%) 0.9061 2.20
39m 35s (- 61m 56s) (11700 39%) 0.6128 1.61
40m 1s (- 61m 44s) (11800 39%) 0.4437 

96m 6s (- 5m 3s) (28500 95%) 1.1944 1.70
96m 31s (- 4m 43s) (28600 95%) 0.5584 1.34
96m 49s (- 4m 23s) (28700 95%) 0.6642 1.20
97m 10s (- 4m 2s) (28800 96%) 1.0403 2.77
97m 31s (- 3m 42s) (28900 96%) 0.8845 1.58
97m 50s (- 3m 22s) (29000 96%) 1.3028 3.07
98m 10s (- 3m 2s) (29100 97%) 0.6293 1.57
98m 30s (- 2m 41s) (29200 97%) 1.1368 2.30
98m 52s (- 2m 21s) (29300 97%) 0.4884 0.98
99m 10s (- 2m 1s) (29400 98%) 0.7699 1.38
99m 33s (- 1m 41s) (29500 98%) 0.8952 2.07
99m 52s (- 1m 20s) (29600 98%) 0.9051 1.45
100m 5s (- 1m 0s) (29700 99%) 0.8347 2.54
100m 27s (- 0m 40s) (29800 99%) 0.3045 1.12
100m 49s (- 0m 20s) (29900 99%) 0.9621 1.73
101m 12s (- 0m 0s) (30000 100%) 1.1221 2.76


In [75]:
torch.save(chatbot.state_dict(), 'C:\\Users\\Jb\\Desktop\\NLP\\saves\\models\\M2DS_2019_final_chatbot_100_75_75_010_16.pth')

In [76]:
# 12596
chatbot.eval()
z = trainer.ErrorCount(chatbot, dialogues_Master_test_var) # erreurs par réponse

The repartition of errors : [12596, 1, 90, 0, 0, 37, 0, 134, 95, 48, 585, 448, 262, 572, 1021, 836, 679, 1168, 113, 363, 158, 285, 0, 8, 85, 118, 46, 44, 3, 3, 0, 0, 0, 0, 0, 16, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 35, 0, 4]


### 6.3.7 Random masking 35 % of corpus words

In [26]:
dialogues_Master_var_035 = variableFromAllDialogues(lang_M2DS, dialogues_Master, rand = 0.35)

In [27]:
chatbot.train()
errors = trainer.train(chatbot, dialogues_Master_var_035, n_iters = 30000, per_dialogue = True, learning_rate=0.005, return_errors = True)

1m 0s (- 299m 54s) (100 0%) 1.1219 2.14
1m 22s (- 203m 46s) (200 0%) 1.3121 2.84
1m 45s (- 174m 4s) (300 1%) 2.0703 4.99
2m 6s (- 155m 51s) (400 1%) 1.4462 2.80
2m 26s (- 144m 3s) (500 1%) 1.4191 3.39
2m 50s (- 139m 6s) (600 2%) 1.4301 3.20
3m 11s (- 133m 25s) (700 2%) 0.7462 1.90
3m 33s (- 129m 47s) (800 2%) 1.2469 3.60
3m 54s (- 126m 9s) (900 3%) 1.4294 2.34
4m 12s (- 122m 9s) (1000 3%) 1.3109 3.33
4m 29s (- 117m 59s) (1100 3%) 0.9693 2.14
4m 49s (- 115m 56s) (1200 4%) 1.4093 2.33
5m 9s (- 113m 51s) (1300 4%) 1.4779 3.61
5m 33s (- 113m 30s) (1400 4%) 0.9846 1.98
5m 52s (- 111m 33s) (1500 5%) 1.5185 3.84
6m 9s (- 109m 19s) (1600 5%) 1.2370 2.26
6m 30s (- 108m 21s) (1700 5%) 1.2372 2.57
6m 51s (- 107m 25s) (1800 6%) 1.4222 2.74
7m 15s (- 107m 14s) (1900 6%) 1.3058 3.05
7m 31s (- 105m 26s) (2000 6%) 0.7979 1.32
7m 50s (- 104m 7s) (2100 7%) 1.2906 3.00
8m 13s (- 103m 56s) (2200 7%) 1.2264 3.87
8m 32s (- 102m 48s) (2300 7%) 1.2675 2.27
8m 51s (- 101m 54s) (2400 8%) 1.2957 2.51
9m 8s (- 10

64m 28s (- 36m 15s) (19200 64%) 1.2385 2.99
64m 46s (- 35m 54s) (19300 64%) 1.0061 2.67
65m 7s (- 35m 34s) (19400 64%) 1.1308 2.16
65m 26s (- 35m 14s) (19500 65%) 1.5347 2.77
65m 45s (- 34m 53s) (19600 65%) 0.8926 2.09
66m 10s (- 34m 36s) (19700 65%) 1.2226 1.63
66m 29s (- 34m 15s) (19800 66%) 1.1477 2.29
66m 51s (- 33m 56s) (19900 66%) 1.4249 2.20
67m 11s (- 33m 35s) (20000 66%) 1.6535 3.28
67m 32s (- 33m 16s) (20100 67%) 1.5029 2.93
67m 54s (- 32m 56s) (20200 67%) 1.3239 3.00
68m 12s (- 32m 35s) (20300 67%) 0.9367 2.19
68m 32s (- 32m 15s) (20400 68%) 1.7937 3.60
68m 55s (- 31m 56s) (20500 68%) 1.3850 2.68
69m 15s (- 31m 36s) (20600 68%) 1.2749 2.71
69m 34s (- 31m 15s) (20700 69%) 1.6588 3.20
69m 55s (- 30m 55s) (20800 69%) 1.4485 2.58
70m 15s (- 30m 35s) (20900 69%) 1.3716 2.07
70m 34s (- 30m 14s) (21000 70%) 0.8272 1.76
70m 56s (- 29m 55s) (21100 70%) 1.3456 2.34
71m 17s (- 29m 35s) (21200 70%) 0.8112 1.59
71m 40s (- 29m 16s) (21300 71%) 1.4894 2.34
72m 1s (- 28m 56s) (21400 71%) 2.

In [28]:
torch.save(chatbot.state_dict(), 'C:\\Users\\Jb\\Desktop\\NLP\\saves\\models\\M2DS_2019_final_chatbot_100_75_75_010_17.pth')

In [29]:
chatbot.eval()
z = trainer.ErrorCount(chatbot, dialogues_Master_test_var) # erreurs par réponse (19854 au total)

The repartition of errors : [12749, 5, 97, 0, 3, 34, 3, 115, 98, 68, 684, 420, 230, 606, 971, 735, 620, 1192, 125, 298, 135, 298, 1, 0, 54, 139, 76, 44, 0, 3, 0, 0, 0, 0, 0, 16, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 35]


In [36]:
chatbot.train()
trainer.train(chatbot, dialogues_Master_var_035, n_iters = 30000, per_dialogue = True, learning_rate=0.005, return_errors = False)

0m 23s (- 115m 14s) (100 0%) 1.4604 2.86
0m 46s (- 114m 24s) (200 0%) 1.1791 3.02
1m 6s (- 110m 1s) (300 1%) 1.1034 2.77
1m 25s (- 105m 55s) (400 1%) 1.2867 2.04
1m 45s (- 104m 4s) (500 1%) 1.1839 1.98
2m 6s (- 102m 56s) (600 2%) 0.6426 1.25
2m 29s (- 104m 3s) (700 2%) 1.3939 2.97
2m 52s (- 105m 1s) (800 2%) 0.8203 1.35
3m 11s (- 103m 19s) (900 3%) 1.1116 2.59
3m 30s (- 101m 38s) (1000 3%) 1.5699 3.34
3m 54s (- 102m 46s) (1100 3%) 0.7033 1.72
4m 14s (- 101m 58s) (1200 4%) 0.9245 1.71
4m 35s (- 101m 17s) (1300 4%) 1.3897 3.10
4m 59s (- 101m 54s) (1400 4%) 0.8956 2.30
5m 19s (- 101m 19s) (1500 5%) 1.0312 1.96
5m 40s (- 100m 52s) (1600 5%) 1.3417 3.00
5m 59s (- 99m 52s) (1700 5%) 1.7007 3.12
6m 19s (- 99m 12s) (1800 6%) 0.8447 1.42
6m 37s (- 98m 2s) (1900 6%) 1.3108 2.86
6m 58s (- 97m 35s) (2000 6%) 1.2724 2.38
7m 17s (- 96m 46s) (2100 7%) 1.0089 1.91
7m 38s (- 96m 33s) (2200 7%) 0.9602 1.62
8m 0s (- 96m 29s) (2300 7%) 1.2330 2.70
8m 19s (- 95m 41s) (2400 8%) 1.2619 2.27
8m 39s (- 95m 18s

64m 43s (- 35m 52s) (19300 64%) 1.2662 3.20
65m 3s (- 35m 33s) (19400 64%) 1.4791 3.34
65m 30s (- 35m 16s) (19500 65%) 1.5583 3.17
65m 59s (- 35m 1s) (19600 65%) 1.0880 1.79
66m 28s (- 34m 45s) (19700 65%) 1.2083 2.46
67m 0s (- 34m 31s) (19800 66%) 1.1357 2.27
67m 34s (- 34m 18s) (19900 66%) 1.6340 4.36
67m 58s (- 33m 59s) (20000 66%) 1.5355 3.06
68m 25s (- 33m 41s) (20100 67%) 1.4226 3.04
68m 55s (- 33m 26s) (20200 67%) 1.0842 2.18
69m 22s (- 33m 9s) (20300 67%) 1.3775 3.27
69m 50s (- 32m 52s) (20400 68%) 0.6888 1.12
70m 24s (- 32m 37s) (20500 68%) 1.6903 4.07
70m 46s (- 32m 17s) (20600 68%) 1.6118 3.09
71m 12s (- 31m 59s) (20700 69%) 1.0288 2.09
71m 38s (- 31m 41s) (20800 69%) 1.1005 2.23
72m 5s (- 31m 23s) (20900 69%) 2.1257 3.70
72m 24s (- 31m 1s) (21000 70%) 1.3964 2.46
72m 47s (- 30m 42s) (21100 70%) 0.9030 2.40
73m 13s (- 30m 23s) (21200 70%) 1.3050 2.13
73m 41s (- 30m 6s) (21300 71%) 1.0606 1.48
74m 5s (- 29m 46s) (21400 71%) 1.0680 2.17
74m 30s (- 29m 27s) (21500 71%) 1.2872 2

In [37]:
torch.save(chatbot.state_dict(), 'C:\\Users\\Jb\\Desktop\\NLP\\saves\\models\\M2DS_2019_final_chatbot_100_75_75_010_18.pth')

In [38]:
chatbot.eval()
z = trainer.ErrorCount(chatbot, dialogues_Master_test_var) # erreurs par réponse

The repartition of errors : [12759, 0, 68, 0, 0, 29, 0, 96, 54, 113, 645, 395, 284, 591, 971, 902, 577, 1111, 136, 330, 143, 281, 0, 0, 67, 108, 47, 43, 1, 35, 0, 0, 0, 0, 0, 16, 0, 5, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 35]


### 6.3.6 Remaining errors

In [47]:
questions_train = list(set([' '.join(pair[0]) for dialogue in dialogues_Master for pair in dialogue]))
len(questions_train)

1364

In [33]:
questions_test = list(set([' '.join(pair[0]) for dialogue in dialogues_Master_test for pair in dialogue]))
len(questions_test)

319

In [51]:
intersect = [el for el in questions_test if el in questions_train]
len(intersect)

6

In [70]:
chatbot.eval()
errors_test = trainer.DialoguesWithErrors(chatbot, dialogues_Master_test_var)
with open(r'C:\\Users\\Jb\\Desktop\\NLP\\saves\\training_errors\\M2DS_2019_final_test_errors.file', 'wb') as fileObject :
    pickle.dump(errors_test, fileObject)

2m 56s (- 26m 28s) (100 10%)
5m 59s (- 23m 59s) (200 20%)
8m 48s (- 20m 33s) (300 30%)
11m 31s (- 17m 17s) (400 40%)
14m 31s (- 14m 31s) (500 50%)
17m 42s (- 11m 48s) (600 60%)
20m 32s (- 8m 48s) (700 70%)
23m 14s (- 5m 48s) (800 80%)
26m 15s (- 2m 55s) (900 90%)
29m 18s (- 0m 0s) (1000 100%)


In [84]:
questions = list(set([' '.join(dialogues_Master_test[k][l][0]) for k in list(errors_test.keys()) for l in errors_test[k]]))
len(questions)

183

In [98]:
questions[23]

'faut il valider toutes les matieres pour valider le master ?'

In [125]:
localisation = {}
for question in questions : localisation[question] = []
for i, dialogue in enumerate(dialogues_Master) :
    for j in range(len(dialogue)) :
        q = ' '.join(dialogues_Master[i][j][0])
        if q in questions : localisation[q].append([i, j])

In [146]:
localisation["faut il valider toutes les matieres pour valider le master ?"]

[]

In [144]:
for key in localisation :
    N = len(localisation[key])
    if N > 0 : print(key, " | nombre d'occurences trouvées : {}".format(N))

je suis de lyon 1  | nombre d'occurences trouvées : 4377
j ' aimerais faire le parcours informatique  | nombre d'occurences trouvées : 8988
je souhaite faire le parcours informatique  | nombre d'occurences trouvées : 7610
je souhaite faire le parcours mathematique  | nombre d'occurences trouvées : 7696
non  | nombre d'occurences trouvées : 13554


In [72]:
extended_errors = {}
for i, dialogue in enumerate(dialogues_Master) :
    sim_errors = [j for j, pair in enumerate(dialogue) if ' '.join(pair[0]) in questions]
    if sim_errors != [] : extended_errors[i] = sim_errors

In [73]:
len(extended_errors)

34768

In [74]:
chatbot.train()
trainer.train(chatbot, dialogues_Master_var_025, n_iters = 10000, learning_rate=0.025, dic = extended_errors)
trainer.train(chatbot, dialogues_Master_var_025, n_iters = 5000, learning_rate=0.025)

0m 18s (- 31m 16s) (100 1%) 0.0041 0.01
0m 36s (- 30m 9s) (200 2%) 0.0001 0.00
0m 54s (- 29m 14s) (300 3%) 0.0001 0.00
1m 11s (- 28m 45s) (400 4%) 0.0002 0.00
1m 30s (- 28m 32s) (500 5%) 0.0001 0.00
1m 47s (- 27m 57s) (600 6%) 0.0001 0.00
2m 5s (- 27m 53s) (700 7%) 0.0001 0.00
2m 23s (- 27m 35s) (800 8%) 0.0001 0.00
2m 42s (- 27m 26s) (900 9%) 0.0001 0.00
2m 59s (- 26m 59s) (1000 10%) 0.0002 0.00
3m 19s (- 26m 52s) (1100 11%) 0.0002 0.00
3m 37s (- 26m 36s) (1200 12%) 0.0002 0.00
3m 57s (- 26m 27s) (1300 13%) 0.0001 0.00
4m 17s (- 26m 21s) (1400 14%) 0.0001 0.00
4m 35s (- 26m 2s) (1500 15%) 0.0002 0.00
4m 54s (- 25m 46s) (1600 16%) 0.0002 0.00
5m 13s (- 25m 29s) (1700 17%) 0.0001 0.00
5m 31s (- 25m 10s) (1800 18%) 0.0001 0.00
5m 49s (- 24m 52s) (1900 19%) 0.0002 0.00
6m 6s (- 24m 27s) (2000 20%) 0.0012 0.02
6m 25s (- 24m 10s) (2100 21%) 0.0003 0.00
6m 44s (- 23m 54s) (2200 22%) 0.0001 0.00
7m 4s (- 23m 41s) (2300 23%) 0.0001 0.00
7m 23s (- 23m 25s) (2400 24%) 0.0062 0.02
7m 43s (- 23m 1

In [132]:
torch.save(chatbot.state_dict(), 'C:\\Users\\Jb\\Desktop\\NLP\\saves\\models\\M2DS_2019_final_chatbot_150_per_dialogue.pth')

## 6.4 test

### 6.4.1 Error counts



In [51]:
chatbot.eval()
z = trainer.ErrorCount(chatbot, dialogues_Master_test_var) # erreurs par réponse

The repartition of errors : [10094, 27, 13, 0, 12, 20, 31, 99, 86, 190, 197, 600, 686, 622, 1292, 1065, 508, 1046, 644, 435, 322, 392, 0, 23, 188, 514, 46, 66, 20, 41, 0, 0, 0, 0, 270, 189, 71, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 44]


In [75]:
chatbot.eval()
z = trainer.ErrorCount(chatbot, dialogues_Master_test_var) # erreurs par réponse

The repartition of errors : [9930, 20, 135, 0, 0, 42, 124, 77, 117, 219, 430, 675, 933, 398, 1878, 605, 897, 972, 677, 499, 149, 293, 2, 9, 215, 258, 2, 56, 0, 34, 0, 0, 0, 0, 12, 82, 71, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 42]


In [52]:
with open(r'C:\\Users\\Jb\\Desktop\\NLP\\saves\\training_errors\\M2DS_2019_final_test_errors.file', 'wb') as fileObject :
    pickle.dump(z, fileObject)

### 6.4.2 Visualization of attention

In [52]:
def showAnswer(model, dialogue):
    answer, log_probas, attn1_w, attn2_w, _ = model.answerTrain(dialogue)
    dialogue = model.flattenDialogue(dialogue)
    print('User : ', ' '.join([model.lang.index2word[int(word)] for word in dialogue[-2][:-1]]))
    print('target : ', ' '.join([model.lang.index2word[int(word)] for word in dialogue[-1][:-1]]))
    print('predic : ', ' '.join([model.lang.index2word[int(word)] for word in answer]))
    return

In [ ]:
# n_errors = 1 : info <-> math
# n_errors = 2 : pas grave
# n_errors = 4 : h par semaises / en tout
# n_errors = 5 : combien de projets
# n_errors = 7 : taux de réussite <-> métier de data scientist / ceremonie / remuneration de stage / rapports
# n_errors = 8 : pas grave
# n_errors = 9 : e-candidat / scolarité
# n_errors = 11 : 'oui , j ' ai etudie a unk_token'
# n_errors = 12 : UNK !


In [53]:
chatbot.eval()
n_errors = 14
i = 724
dialogue = dialogues_Master_test_var[z[n_errors][i][0]][:z[n_errors][i][1]+1]
#chatbot.showAttention(dialogue)
showAnswer(chatbot, dialogue)

User :  oui , j ' ai etudie a unk_token
target :  etes - vous un etudiant inscrit a lyon 1 ?
predic :  il faut que vous deposiez votre candidature directement aupres de campus france .


In [37]:
chatbot.eval()
i = 10
dialogue = dialogues_Master_test_var[i][:-3]
chatbot.showAttention(dialogue)

ValueError: too many values to unpack (expected 4)

### 6.4.3 Interactive mode

write **quit** or **q** or **eoc** (end of conversation) to close conversation interface :

In [54]:
def ReplaceMotVar(motsVar, raw_sentence):
    sentence = []
    word_list = raw_sentence.split(' ')
    for word in word_list :
        if word in motsVar.keys() :
            sentence.append(motsVar[word])
        else :
            sentence.append(word)
    return ' '.join(sentence)


def repair(sentence) :
    s = re.sub(" ' ", "'", sentence)
    s = re.sub(" - ", "-", s)
    s = re.sub(" ,", ",", s)
    s = re.sub(r'(?<=\d) \. (?=\d)', '.', s)
    s = re.sub(" \.", ".", s)
    s = re.sub("\( ", "(", s)
    s = re.sub(" \)", ")", s)
    s = s[0].upper() + s[1:]
    return s


def InteractiveEvaluation(agent) :
    agent.initMemory()
    while True :
        text = input()
        #print('User : {}'.format(text))
        if text == 'eoc' or text == 'q' or text == 'quit' : 
            break
        reponse, attn1_weights, attn2_weights = agent(text)
        reponse = ReplaceMotVar(motsVar, reponse)
        reponse = repair(reponse)
        print('Bot  : {}'.format(reponse))

In [128]:
chatbot.eval()
InteractiveEvaluation(chatbot)

comment on peut payer ?
Bot  : Vous pouvez payer par carte, par cheque bancaire, par virement ou en espece.
quels sont les modes de payement
Bot  : Le stage doit avoir une duree de 4 a 6 mois.
combien y a - t - il d ' ects par cours ?
Bot  : Il y a en tout 400h de cours.
combien d'etcs
Bot  : Il y a en tout 400h de cours.
le nombre d'etcs
Bot  : D'accord, comment puis-je vous aider ?
le nombre d etcs est de combien ?
Bot  : Je suis la pour vous renseigner sur le master 2 data science.
etcs
Bot  : Avez-vous d'autres questions ?
oui, est ce que c'est difficile ?
Bot  : Non, mais il faut travailler assidument.
ca veur dire qu'il faut travailler jours et nuits ?
Bot  : Il faut que vous deposiez votre candidature directement aupres de campus france.
quit


In [66]:
chatbot.eval()
InteractiveEvaluation(chatbot)

salut chatbot
Bot  : Bonjour, comment puis-je vous aider ?
tu sais dire quoi ?
Bot  : Excusez-moi, accordez-moi une seconde chance.
est ce que je peux encore m'inscire
Bot  : Le master est moitie mathematiques moitie informatique.
mais est il encore possible de s'inscrire
Bot  : Le calendrier des candidatures est disponible sur la plateforme e-candidat, onglet "offre de formation".
le lien stp
Bot  : La rentree est le 5 septembre 2018
tks mofucker
Bot  : Bonjour, comment puis-je vous aider ?
quit


# 7 Noise filtering

## 7.1 Noisy data generation

In [129]:
random.seed(42)

In [130]:
def randomMask(clean_sentences, rand = 0.2) :
    def choseWord(word) :
        return word if random.random() >= rand else 'UNK'
    masked_sentences = []
    for sentence in clean_sentences :
        words = [choseWord(word) for word in sentence.split()]
        masked_sentences.append(' '.join(words))
    return masked_sentences



def randomNoise(clean_sentences, lang, prop = 1.0):
    noisy_sentences = []
    for sentence in clean_sentences :
        if len(sentence.split()) >= 5 :
            words = [lang.index2word[random.randint(3, lang.n_words-1)] for _ in sentence.split()]
            words = [word for i, word in enumerate(words) if i < 5 or random.random() < prop]
            rand_unk = random.randint(1, len(words)-1)
            words.insert(rand_unk, 'UNK')
            if rand_unk > len(words)/ 2 : words += random.choice(['?', '.'])
            noisy_sentences.append(' '.join(words))
    return noisy_sentences

In [131]:
clean_sentences = [sentence for sentence in list(set([' '.join(pair[0]) for diag in dialogues_Master for pair in diag]))]
print(len(clean_sentences))

masked_sentences_000 = []
masked_sentences_015 = []
masked_sentences_025 = []
noisy_sentences_100 = []
noisy_sentences_050 = []

for _ in range(10) :
    masked_sentences_000 += clean_sentences
    masked_sentences_015 += randomMask(clean_sentences, rand = 0.15)
    masked_sentences_025 += randomMask(clean_sentences, rand = 0.25)

    noisy_sentences_100 += randomNoise(clean_sentences, lang_M2DS, prop = 1.0)
    noisy_sentences_050 += randomNoise(clean_sentences, lang_M2DS, prop = 0.5)

1364


In [134]:
discriminator_dataset = [[sentence, 1] for sentence in masked_sentences_000 + masked_sentences_015 + masked_sentences_025] + \
                        [[sentence, 0] for sentence in noisy_sentences_100 + noisy_sentences_050]
random.shuffle(discriminator_dataset)
len(discriminator_dataset)

66900

In [135]:
discriminator_dataset[7][0]

"qui enseigne UNK ' ue big data UNK UNK"

## 7.2 Noise filtering model

In [136]:
from chatNLP.misc.Noise_Filter import NoiseFilter, NoiseFilterTrainer, NoiseFilterWrapper

In [138]:
noise_filter = NoiseFilter(chatbot, pretrained = True, layers = [50, 50]).to(device)

In [139]:
noise_filter.nbParametres()

12702

In [ ]:
#noise_filter.load_state_dict(torch.load('C:\\Users\\Jb\\Desktop\\NLP\\saves\\models\\M2DS_2019_final_noise_filter.pth'))

In [140]:
noise_filter_trainer = NoiseFilterTrainer(device,
                                          criterion = nn.NLLLoss(), 
                                          optimizer = optim.SGD)

In [141]:
noise_filter.train()
noise_filter_trainer.train(noise_filter, sentences = discriminator_dataset, n_iters = 20000, learning_rate=0.01)

NameError: name 'time' is not defined

In [ ]:
noise_filter.train()
noise_filter_trainer.train(noise_filter, sentences = discriminator_dataset, n_iters = 10000, learning_rate=0.005)
noise_filter_trainer.train(noise_filter, sentences = discriminator_dataset, n_iters = 10000, learning_rate=0.0025)

In [ ]:
noise_filter.train()
noise_filter_trainer.train(noise_filter, sentences = discriminator_dataset, n_iters = 10000, learning_rate=0.001)

In [ ]:
torch.save(noise_filter.state_dict(), 'C:\\Users\\Jb\\Desktop\\NLP\\saves\\models\\M2DS_2019_final_noise_filter.pth')

## 7.3 Noise filtering integration

model built on top of a pretrained chatbot model

In [ ]:
def SmartInteractiveEvaluation(agent) :
    agent.chatbot.initMemory()
    while True :
        text = input()
        #print('User : {}'.format(text))
        if text == 'eoc' or text == 'q' or text == 'quit' : 
            break
        reponse, attn1_weights, attn2_weights = agent(text)
        reponse = ReplaceMotVar(motsVar, reponse)
        reponse = repair(reponse)
        print('Bot  : {}'.format(reponse))

In [ ]:
noise_filter.eval()
smart_chatbot = NoiseFilterWrapper(noise_filter, chatbot)
smart_chatbot.eval()
SmartInteractiveEvaluation(smart_chatbot)